In [1]:
import numpy as np
import pandas as pd
import anndata as ad
import pickle
import itertools
import networkx as nx
import scanpy as sc
import matplotlib.pyplot as plt
import pwlf

import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '/Users/yijwang-admin/Documents/Research/GRN_PPI/L0GL_PPI_GRN/')

from minConf.minConf_PQN import minConF_PQN

# Functions

In [2]:


def ShorestPath_GivenTFs(Gnx, TFlist, FilePathes):
    Ol_TF = list(set(Gnx.nodes) & set(TFlist))
    #print("Only ", len(Ol_TF), " TFs (", len(TFlist), " in total) appear in the network.")
    
    fh = open(FilePathes, 'w')
    ShortPath = list()
    NumNoPath = 0
    for i in range(len(Ol_TF)):
        for j in range(i+1, len(Ol_TF)):
            if nx.has_path(Gnx, Ol_TF[i], Ol_TF[j]):
                sp = nx.shortest_path_length(Gnx, Ol_TF[i], Ol_TF[j])
                Path = nx.shortest_path(Gnx, source=Ol_TF[i], target=Ol_TF[j])
                for n in Path:
                    fh.write("%s " % n)
                fh.write("%d\n" % sp)
                ShortPath.append(sp)
            else:
                fh.write("No path between %s and %s\n" % ( Ol_TF[i], Ol_TF[j]))
                NumNoPath = NumNoPath + 1
                #ShortPath.append(10)
    fh.close()

    return ShortPath

def Short_Path_Matrix(Gnx, TFlist, MaxLen = 6):
    NodesG = Gnx.nodes
    
    N = len(TFlist)
    Smat = np.zeros((N,N))
    
    for i in range(N):
        for j in range(i+1, N):
            if TFlist[i] in NodesG and TFlist[j] in NodesG:
                if nx.has_path(Gnx, TFlist[i], TFlist[j]):
                    Smat[i,j] = nx.shortest_path_length(Gnx, TFlist[i], TFlist[j])
                    Smat[j,i] = Smat[i,j]
                else:
                    Smat[i,j] = MaxLen
                    Smat[j,i] = Smat[i,j]
            else:
                Smat[i,j] = MaxLen
                Smat[j,i] = Smat[i,j]
                
    return Smat


def Short_Path_DSD_Matrix(Gnx, TFlist, MaxLen = 5):
    NodesG = Gnx.nodes
    
    N = len(TFlist)
    Smat = np.zeros((N,N))
    
    for i in range(N):
        for j in range(i+1, N):
            if TFlist[i] in NodesG and TFlist[j] in NodesG:
                Smat[i,j] = Gnx.edges[TFlist[i], TFlist[j]]['weight']
                Smat[j,i] = Smat[i,j]   
                #print(Smat[i,j])
            else:
                #print(TFlist[i], TFlist[j])
                Smat[i,j] = MaxLen
                Smat[j,i] = Smat[i,j]
                
    return Smat



In [3]:
# 

import numpy as np
from scipy.sparse import spdiags
from scipy.linalg import inv
from gurobipy import Model, GRB, QuadExpr

def L0GLObj(u, X, y, pho):
    # u (feature, 1)
    # X (instance, feature)
    # y (instance, 1)
    # pho (1, 1)
    #print(pho)
    #print(u)
    n, d = X.shape
    SpDiag = spdiags(u.flatten(), 0, d, d)
    # print(SpDiag)
    # print(f"SpDiag: {SpDiag.toarray()[-1][-1]}")
    M = inv((1/pho) * X @ SpDiag @ X.conj().T + np.eye(n))
    f = y.conj().T @ M @ y

    g = -(1/pho) * ((X.conj().T @ M @ y)**2)

    return f, g

def L0Obj(u, X, y, pho):
    # u (feature, 1)
    # X (instance, feature)
    # y (instance, 1)
    # pho (1, 1)
    #print(pho)
    n, d = X.shape
    SpDiag = spdiags(u.flatten(), 0, d, d)
    # print(SpDiag)
    # print(f"SpDiag: {SpDiag.toarray()[-1][-1]}")
    M = inv((1/pho) * X @ SpDiag @ X.conj().T + np.eye(n))
    f = y.conj().T @ M @ y

    g = -(1/pho) * ((X.conj().T @ M @ y)**2)

    return f, g

def L0GL_PPI_Obj(u, X, y, S, pho, lam):
    # u (feature, 1)
    # X (instance, feature)
    # y (instance, 1)
    # pho (1, 1)
    #print(pho)
    #print(u)
    n, d = X.shape
    SpDiag = spdiags(u.flatten(), 0, d, d)
    # print(SpDiag)
    # print(f"SpDiag: {SpDiag.toarray()[-1][-1]}")
    M = inv((1/pho) * X @ SpDiag @ X.T + np.eye(n))
    f = y.T @ M @ y + lam * u.T @ S @ u

    g = -(1/pho) * ((X.T @ M @ y)**2) + 2 * lam * S @ u

    return f, g


def ProjCSimplex(u, k):
    """
    Projects vector u onto the simplex defined by the sum of elements equal to k.
    """
    # u = u.flatten()
    # print(u.shape)
    n, m = u.shape
    # print(f"n: {n}, {m}")

    H = 0.5*np.eye(n)
    f = -0.5 * u
    A = np.ones((1, n))
    b = k

    lb = np.zeros((n, 1))
    ub = np.ones((n, 1))
    
    if np.sum(u) < k and np.all(np.logical_and(u >= 0, u <= 1)):
        up = u
    else:
        d = len(u)
        e = np.ones((d, 1))
        z0 = np.zeros((d, 1))
        l0 = float(0 + np.random.rand() * np.min(u))
        error = 100
        idt = 0
        while error > 1e-8:
            tmp = u - l0
            # tmp[tmp > 1] = 1
            # tmp[tmp < 0] = 0
            tmp = np.clip(tmp, 0, 1)
            error = k - np.sum(tmp)
            n = np.count_nonzero(tmp)
            if n == 0:
                l0 = float(0+np.random.rand() * np.min(u))
                error = 100
                continue
            idt += 1
            l1 = l0 - (error / n)
            tmp = u - l1
            # tmp[tmp > 1] = 1
            # tmp[tmp < 0] = 0
            tmp = np.clip(tmp, 0, 1)
            error = np.abs(k - np.sum(tmp))
            l0 = l1
        
        if l0 < 0:
            l0 = 0
        tmp = u - l0
        # tmp[tmp > 1] = 1
        # tmp[tmp < 0] = 0
        tmp = np.clip(tmp, 0, 1)
        up = tmp

    # print(f"up: {up[-1][-1]}")
    
    return up

def ProjCSimplexGL_Gurobi(u, k, Group, h):
    n, m = u.shape
    g = len(Group)
    
    # Create matrices H1 and H2
    H1 = np.eye(n)
    f1 = -2*u.flatten()

    H2 = np.zeros((g, g))
    f2 = np.zeros(g)

    H = np.block([[H1, np.zeros((n, g))], [np.zeros((g, n)), H2]])
    f = np.concatenate([f1, f2])
    #print(u.shape)
    #print(f.shape)

    # Constructing constraint matrices A1 and A2
    A1 = np.ones((1, n))
    A2 = np.zeros((1, g))
    gcn = 0

    for i in range(g):
        for j in range(len(Group[i])):
            A1t = np.zeros((1, n))
            A1t[0, Group[i][j]] = 1
            A1 = np.vstack([A1, A1t])
            
            A2t = np.zeros((1, g))
            A2t[0, i] = -1
            A2 = np.vstack([A2, A2t])
            gcn += 1

    A1 = np.vstack([A1, np.zeros((1, n))])
    A2 = np.vstack([A2, np.ones((1, g))])
    A = np.hstack([A1, A2])

    # RHS vector
    b = np.zeros(gcn + 2)
    b[0] = k
    b[-1] = h

    # Lower and upper bounds
    lb = np.zeros(n + g)
    ub = np.ones(n + g)

    # Setup Gurobi model
    model = Model()

    # Add variables
    Q = H
    x = model.addMVar(n + g, ub=1.0, lb=0.0)
    
    model.setObjective(x@Q@x+ x@f)
    model.addConstr(A @ x <= b)
    

    # Set Gurobi parameters
    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 500)

    # Optimize model
    model.optimize()

    # Get the results
    up = x.x[:n]
    zp = x.x[n:]

    #print(up.shape)
    return up[:,np.newaxis]

# used for PQN,comparing to ProjCSimplexGL_Gurobi_LB, it has two output
def ProjCSimplexGL_Gurobi_v2(u, k, Group, h):
    n, m = u.shape
    g = len(Group)
    
    # Create matrices H1 and H2
    H1 = np.eye(n)
    f1 = -2*u.flatten()

    H2 = np.zeros((g, g))
    f2 = np.zeros(g)

    H = np.block([[H1, np.zeros((n, g))], [np.zeros((g, n)), H2]])
    f = np.concatenate([f1, f2])
    #print(u.shape)
    #print(f.shape)

    # Constructing constraint matrices A1 and A2
    A1 = np.ones((1, n))
    A2 = np.zeros((1, g))
    gcn = 0

    for i in range(g):
        for j in range(len(Group[i])):
            A1t = np.zeros((1, n))
            A1t[0, Group[i][j]] = 1
            A1 = np.vstack([A1, A1t])
            
            A2t = np.zeros((1, g))
            A2t[0, i] = -1
            A2 = np.vstack([A2, A2t])
            gcn += 1

    A1 = np.vstack([A1, np.zeros((1, n))])
    A2 = np.vstack([A2, np.ones((1, g))])
    A = np.hstack([A1, A2])

    # RHS vector
    b = np.zeros(gcn + 2)
    b[0] = k
    b[-1] = h

    # Lower and upper bounds
    lb = np.zeros(n + g)
    ub = np.ones(n + g)

    # Setup Gurobi model
    model = Model()

    # Add variables
    Q = H
    x = model.addMVar(n + g, ub=1.0, lb=0.0)
    
    model.setObjective(x@Q@x+ x@f)
    model.addConstr(A @ x <= b)
    

    # Set Gurobi parameters
    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 500)

    # Optimize model
    model.optimize()

    # Get the results
    up = x.x[:n]
    zp = x.x[n:]

    #print(up.shape)
    return up[:,np.newaxis], zp[:,np.newaxis]

# used for project, comparing to ProjCSimplexGL_Gurobi_v2, it only has one output
def ProjCSimplexGL_Gurobi_LB(u, k, Group, h):
    n, m = u.shape
    g = len(Group)
    
    # Create matrices H1 and H2
    H1 = np.eye(n)
    f1 = -2*u.flatten()

    H2 = np.zeros((g, g))
    f2 = np.zeros(g)

    H = np.block([[H1, np.zeros((n, g))], [np.zeros((g, n)), H2]])
    f = np.concatenate([f1, f2])
    #print(u.shape)
    #print(f.shape)

    # Constructing constraint matrices A1 and A2
    A1 = np.ones((1, n))
    A1t = -np.ones((1, n))
    A1 = np.vstack([A1, A1t])
    A2 = np.zeros((2, g))
    gcn = 0

    for i in range(g):
        for j in range(len(Group[i])):
            A1t = np.zeros((1, n))
            A1t[0, Group[i][j]] = 1
            A1 = np.vstack([A1, A1t])
            
            A2t = np.zeros((1, g))
            A2t[0, i] = -1
            A2 = np.vstack([A2, A2t])
            gcn += 1

    A1 = np.vstack([A1, np.zeros((1, n))])
    A2 = np.vstack([A2, np.ones((1, g))])
    A = np.hstack([A1, A2])

    # RHS vector
    b = np.zeros(gcn + 3)
    b[0] = k
    b[1] = -k+1
    b[-1] = h

    # Lower and upper bounds
    lb = np.zeros(n + g)
    ub = np.ones(n + g)

    # Setup Gurobi model
    model = Model()

    # Add variables
    Q = H
    x = model.addMVar(n + g, ub=1.0, lb=0.0)
    
    model.setObjective(x@Q@x+ x@f)
    model.addConstr(A @ x <= b)
    

    # Set Gurobi parameters
    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 500)

    # Optimize model
    model.optimize()

    #print(x)
    # Get the results
    up = x.x[:n]
    zp = x.x[n:]

    #print(up.shape)
    return up[:,np.newaxis]

def ProjCSimplexGL_Gurobi_LB_v2(u, k, Group, h):
    n, m = u.shape
    g = len(Group)
    
    # Create matrices H1 and H2
    H1 = np.eye(n)
    f1 = -2*u.flatten()

    H2 = np.zeros((g, g))
    f2 = np.zeros(g)

    H = np.block([[H1, np.zeros((n, g))], [np.zeros((g, n)), H2]])
    f = np.concatenate([f1, f2])
    #print(u.shape)
    #print(f.shape)

    # Constructing constraint matrices A1 and A2
    A1 = np.ones((1, n))
    A1t = -np.ones((1, n))
    A1 = np.vstack([A1, A1t])
    A2 = np.zeros((2, g))
    gcn = 0

    for i in range(g):
        for j in range(len(Group[i])):
            A1t = np.zeros((1, n))
            A1t[0, Group[i][j]] = 1
            A1 = np.vstack([A1, A1t])
            
            A2t = np.zeros((1, g))
            A2t[0, i] = -1
            A2 = np.vstack([A2, A2t])
            gcn += 1

    A1 = np.vstack([A1, np.zeros((1, n))])
    A2 = np.vstack([A2, np.ones((1, g))])
    A = np.hstack([A1, A2])

    # RHS vector
    b = np.zeros(gcn + 3)
    b[0] = k
    b[1] = -k+1
    b[-1] = h

    # Lower and upper bounds
    lb = np.zeros(n + g)
    ub = np.ones(n + g)

    # Setup Gurobi model
    model = Model()

    # Add variables
    Q = H
    x = model.addMVar(n + g, ub=1.0, lb=0.0)
    
    model.setObjective(x@Q@x+ x@f)
    model.addConstr(A @ x <= b)
    

    # Set Gurobi parameters
    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 500)

    # Optimize model
    model.optimize()

    # Get the results
    up = x.x[:n]
    zp = x.x[n:]

    #print(up.shape)
    return up[:,np.newaxis], zp[:,np.newaxis]

In [4]:
def Get_TFs_in_Data(TargetGene, scATAC_df, TF_Col, Gene_GTOpenR_Mapping, Gene_in_Data):
    GL = [TargetGene]
    
    # get df for the target gene
    Tmp_Input = scATAC_df[scATAC_df['Genes_within_250k'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
    
    TFList_Raw = Tmp_Input[TF_Col].tolist()
    Chr_Position_Raw = Tmp_Input['chr_position'].tolist()
    TF_Groups_in_Data = list()
    count = 0
    for tfl, chp in zip(TFList_Raw, Chr_Position_Raw):
        if chp in Map_Gene_GTOpenR[GL[0]]:
            hit = 1
        else:
            hit = 0
        olp = list(set(tfl) & set(Gene_in_Data))
        TF_Groups_in_Data.append(olp)
        print(chp, count, olp, hit)
        count = count + 1
        
    return TF_Groups_in_Data, Tmp_Input

def Get_TFs_in_Data_noPrint(TargetGene, scATAC_df, TF_Col, Gene_GTOpenR_Mapping, Gene_in_Data):
    GL = [TargetGene]
    
    # get df for the target gene
    Tmp_Input = scATAC_df[scATAC_df['Genes_within_250k'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
    
    TFList_Raw = Tmp_Input[TF_Col].tolist()
    Chr_Position_Raw = Tmp_Input['chr_position'].tolist()
    TF_Groups_in_Data = list()
    count = 0
    for tfl, chp in zip(TFList_Raw, Chr_Position_Raw):
        if chp in Map_Gene_GTOpenR[GL[0]]:
            hit = 1
        else:
            hit = 0
        olp = list(set(tfl) & set(Gene_in_Data))
        TF_Groups_in_Data.append(olp)
        #print(count, olp, hit)
        count = count + 1
        
    return TF_Groups_in_Data, Tmp_Input



def Show_TFs_in_Data(TargetGene, scATAC_df, TF_Col, Gene_GTOpenR_Mapping, Pred_TFs):
    GL = [TargetGene]
    
    # get df for the target gene
    Tmp_Input = scATAC_df[scATAC_df['Genes_within_250k'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
    
    TFList_Raw = Tmp_Input[TF_Col].tolist()
    Chr_Position_Raw = Tmp_Input['chr_position'].tolist()
    TF_Groups_in_Data = list()
    count = 0
    for tfl, chp in zip(TFList_Raw, Chr_Position_Raw):
        if chp in Map_Gene_GTOpenR[GL[0]]:
            hit = 1
        else:
            hit = 0
        olp = list(set(tfl) & set(Pred_TFs))
        TF_Groups_in_Data.append(olp)
        print(count, olp, hit)
        count = count + 1
        
    return TF_Groups_in_Data


def Group_Ind(TFinData, TF_Groups_in_Data):
    Map_TF_Ind = {k: v for v, k in enumerate(TFinData)}
    
    GroupT = list()
    for tfl in TF_Groups_in_Data:
        if len(tfl) == 0:
            continue
        tmplist = list()
        for tf in tfl:
            tmplist.append(Map_TF_Ind[tf])
        GroupT.append(tmplist)
        
    return GroupT

def Extract_Solution(u_output, z_output, group_info, k_input, h_input):
    zsort_h = np.argsort(-z_output.flatten())[0:h_input]
    #print("Selected group index:", zsort_h)
    
    Idt_ = list()
    for ht in zsort_h:
        Idt_.extend(group_info[ht])
    Idt_ = np.array(Idt_)
    #print(Idt_)
        
    u_select = u_output.flatten()[Idt_]
    Idt_u = np.argsort(-u_select.flatten())[0:k_input]
    
    #print(u_select[Idt_u])
    
    final_idt = Idt_[Idt_u]
    
    return final_idt

def Split_Sample(X_raw, y_raw, SID, percentage_training):
    NumSample, NumFeature = X_raw.shape
    #print("Number of features:", NumFeature)

    RandInd = np.random.permutation(NumSample)
    NumTrainSample = int(percentage_training*NumSample)
    TrainSampleInd = RandInd[0:NumTrainSample]
    OutSampleInd = RandInd[NumTrainSample:]
    #print("Train sample #", len(TrainSampleInd), "Out-of-sample #", len(OutSampleInd))

    X_train = X_raw[TrainSampleInd,:]
    X_outsample = X_raw[OutSampleInd,:] 
    #print(X.shape)
    #print(X_outsample.shape)

    y_train = y_raw[TrainSampleInd]
    y_outsample = y_raw[OutSampleInd]
    
    SID_train = list( SID[i] for i in TrainSampleInd)
    SID_outsample = list( SID[i] for i in OutSampleInd)

    return X_train, y_train, X_outsample, y_outsample, SID_train, SID_outsample

def Split_Sample_scRNA(scRNA_Anndata, percentage_training):
    NumSample, NumFeature = scRNA_Anndata.shape
    SID = scRNA_Anndata.obs.index.tolist()
    
    RandInd = np.random.permutation(NumSample)
    NumTrainSample = int(percentage_training*NumSample)
    TrainSampleInd = RandInd[0:NumTrainSample]
    OutSampleInd = RandInd[NumTrainSample:]
    
    SID_train = list( SID[i] for i in TrainSampleInd)
    SID_outsample = list( SID[i] for i in OutSampleInd)
    
    scRNA_Anndata_Train = scRNA_Anndata[TrainSampleInd,:]
    scRNA_Anndata_OutSample = scRNA_Anndata[OutSampleInd,:] 
    
    return scRNA_Anndata_Train, scRNA_Anndata_OutSample, SID_train, SID_outsample
    



In [5]:
def DF_Column_to_List(DataFrame, ColName):
    return list(set(list(itertools.chain.from_iterable(DataFrame[ColName].tolist()))))

def Split_Sample_scRNA(scRNA_Anndata, percentage_training):
    NumSample, NumFeature = scRNA_Anndata.shape
    SID = scRNA_Anndata.obs.index.tolist()
    
    RandInd = np.random.permutation(NumSample)
    NumTrainSample = int(percentage_training*NumSample)
    TrainSampleInd = RandInd[0:NumTrainSample]
    OutSampleInd = RandInd[NumTrainSample:]
    
    SID_train = list( SID[i] for i in TrainSampleInd)
    SID_outsample = list( SID[i] for i in OutSampleInd)
    
    scRNA_Anndata_Train = scRNA_Anndata[TrainSampleInd,:]
    scRNA_Anndata_OutSample = scRNA_Anndata[OutSampleInd,:] 
    
    return scRNA_Anndata_Train, scRNA_Anndata_OutSample, SID_train, SID_outsample

def Get_TFs_with_Exp(TargetGene, scATAC_df, TF_Col, Gene_in_scRNA):
    GL = [TargetGene]
    
    # get df for the target gene
    Tmp_Input = scATAC_df[scATAC_df['Genes_within_250k'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
    
    TFList_All = list(set(itertools.chain.from_iterable(Tmp_Input[TF_Col].tolist())))
    
    TFList_with_Exp = list(set(TFList_All) & set(Gene_in_scRNA))
        
    return TFList_with_Exp

def Short_Path_DSD_Matrix(Gnx, TFlist, MaxLen = 5):
    NodesG = Gnx.nodes
    
    N = len(TFlist)
    Smat = np.zeros((N,N))
    
    for i in range(N):
        for j in range(i+1, N):
            if TFlist[i] in NodesG and TFlist[j] in NodesG:
                Smat[i,j] = Gnx.edges[TFlist[i], TFlist[j]]['weight']
                Smat[j,i] = Smat[i,j]   
                #print(Smat[i,j])
            else:
                #print(TFlist[i], TFlist[j])
                Smat[i,j] = MaxLen
                Smat[j,i] = Smat[i,j]
                
    return Smat

def Out_of_Sample_MSE(scRNA, TG, TFList, SID_train, SID_outsample, pho):
    XSelect = scRNA[SID_train, TFList].X
    y = scRNA[SID_train, TG].X
    w_est = np.linalg.pinv(XSelect.T @ XSelect + pho*np.eye(len(TFList))) @ XSelect.T @ y
    #print(TF_Select)
    #print(np.abs(w_est.flatten()))

    X_outsample_select = scRNA[SID_outsample, TFList].X
    y_outsample = scRNA[SID_outsample, TG].X
    MSE = (1. / len(SID_outsample)) * np.linalg.norm(y_outsample - X_outsample_select @ w_est)
    
    return MSE

In [6]:
def ProjCSimplexGL_Gurobi_LB_v2(u, k, Group, h):
    n, m = u.shape
    g = len(Group)
    
    # Create matrices H1 and H2
    H1 = np.eye(n)
    f1 = -2*u.flatten()

    H2 = np.zeros((g, g))
    f2 = np.zeros(g)

    H = np.block([[H1, np.zeros((n, g))], [np.zeros((g, n)), H2]])
    f = np.concatenate([f1, f2])
    #print(u.shape)
    #print(f.shape)

    # Constructing constraint matrices A1 and A2
    A1 = np.ones((1, n))
    A1t = -np.ones((1, n))
    A1 = np.vstack([A1, A1t])
    A2 = np.zeros((2, g))
    gcn = 0

    for i in range(g):
        for j in range(len(Group[i])):
            A1t = np.zeros((1, n))
            A1t[0, Group[i][j]] = 1
            A1 = np.vstack([A1, A1t])
            
            A2t = np.zeros((1, g))
            A2t[0, i] = -1
            A2 = np.vstack([A2, A2t])
            gcn += 1

    A1 = np.vstack([A1, np.zeros((1, n))])
    A2 = np.vstack([A2, np.ones((1, g))])
    A = np.hstack([A1, A2])

    # RHS vector
    b = np.zeros(gcn + 3)
    b[0] = k
    b[1] = -k+1
    b[-1] = h

    # Lower and upper bounds
    lb = np.zeros(n + g)
    ub = np.ones(n + g)

    # Setup Gurobi model
    model = Model()

    # Add variables
    Q = H
    x = model.addMVar(n + g, ub=1.0, lb=0.0)
    
    model.setObjective(x@Q@x+ x@f)
    model.addConstr(A @ x <= b)
    

    # Set Gurobi parameters
    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 500)

    # Optimize model
    model.optimize()

    # Get the results
    up = x.x[:n]
    zp = x.x[n:]

    #print(up.shape)
    return up[:,np.newaxis], zp[:,np.newaxis]

def Extract_Solution(u_output, z_output, group_info, k_input, h_input):
    zsort_h = np.argsort(-z_output.flatten())[0:h_input]
    #print("Selected group index:", zsort_h)
    
    Idt_ = list()
    for ht in zsort_h:
        Idt_.extend(group_info[ht])
    Idt_ = np.array(Idt_)
    #print(Idt_)
        
    u_select = u_output.flatten()[Idt_]
    Idt_u = np.argsort(-u_select.flatten())[0:k_input]
    
    #print(u_select[Idt_u])
    
    final_idt = Idt_[Idt_u]
    
    return final_idt


def Split_Sample_scRNA(scRNA_Anndata, percentage_training):
    NumSample, NumFeature = scRNA_Anndata.shape
    SID = scRNA_Anndata.obs.index.tolist()
    
    RandInd = np.random.permutation(NumSample)
    NumTrainSample = int(percentage_training*NumSample)
    TrainSampleInd = RandInd[0:NumTrainSample]
    OutSampleInd = RandInd[NumTrainSample:]
    
    SID_train = list( SID[i] for i in TrainSampleInd)
    SID_outsample = list( SID[i] for i in OutSampleInd)
    
    scRNA_Anndata_Train = scRNA_Anndata[TrainSampleInd,:]
    scRNA_Anndata_OutSample = scRNA_Anndata[OutSampleInd,:] 
    
    return scRNA_Anndata_Train, scRNA_Anndata_OutSample, SID_train, SID_outsample

def Find_best_k(KRange, MES_Array):
    
    best_k = -1
    
    if (MES_Array[0] - MES_Array[-1]) / MES_Array[0] <= 0.3:
        return best_k
    else:
        my_pwlf = pwlf.PiecewiseLinFit(KRange, MES_Array)
        breaks = my_pwlf.fit(2)
        best_k = np.ceil(breaks[1])
        return int(best_k)
        

def Run_GRIPNet(scRNA_Anndata, SID_train, SID_outsample, scATAC_DF, DisMat_PPI, TF_Motif_Metric, TargetGenes, TFNum_Range = range(3,11), Lambda = 0.0001, Pho = 0.0001): 
    
    Genes_with_Exp = scRNA_Anndata[SID_train,:].var.index.tolist()
    
    GRIPNet_DF = pd.DataFrame(columns=['TG', 'TFs', 'Out_of_Sample MSE'])
    count = 1
    for tg in TargetGenes:
        if tg not in Genes_with_Exp:
            print("No expression data for ", tg)
            continue
            
        # Collect all TFs (with gene expression), whose motifs matched to the open regions within 250kb around TG and 
        TFs_for_TG = Get_TFs_with_Exp(tg, scATAC_DF, TF_Motif_Metric, Genes_with_Exp)
        #print(TFs_for_TG)
        
        if len(TFs_for_TG) <= 3:
            count = count + 1
            continue
        
        # Parameters setup for the algo.
        NumFeature = len(TFs_for_TG)
        NumSample = scRNA_Anndata[:,TFs_for_TG].X.shape[0]
        
        # Regression data
        X = scRNA_Anndata[SID_train,TFs_for_TG].X
        y = scRNA_Anndata[SID_train,tg].X
        uSimplex = np.ones((NumFeature, 1)) * (1 / NumFeature)
        
        # Distance between poential TFs
        S =  Short_Path_DSD_Matrix(DisMat_PPI, TFs_for_TG)
        #print(S.shape)
      
        Group = [range(NumFeature)]
        h = 1
        lamb = Lambda
        pho = Pho

        if len(TFs_for_TG) < TFNum_Range[-1]:
            K_Range = range(3, len(TFs_for_TG)+1)
        else:
            K_Range = TFNum_Range
        MSE_ = np.zeros(len(K_Range))
        TF_Select_Collect = {}
        Map_Ind_k = {}
        for i in range(len(K_Range)):
            k = K_Range[i]
    
            #print(lamb)
            # Set up Objective Function
            funObj_PPI = lambda w: L0GL_PPI_Obj(w, X, y, S, pho, lamb)
            # Set up Simplex Projection Function
            funProj_GL_LB = lambda w: ProjCSimplexGL_Gurobi_LB(w, k, Group, h)
            # parameters that can be changed
            options = {'maxIter': 400, 'verbose': -1}
            
            uout, obj, _ = minConF_PQN(funObj_PPI, uSimplex, funProj_GL_LB, options)
    
            uout_ppi, zout_ppi = ProjCSimplexGL_Gurobi_LB_v2(uout, k, Group, h)
            SelectID = Extract_Solution(uout_ppi, zout_ppi, Group, k, h)

            TF_Select = list()
            for idt in SelectID:
                TF_Select.append(TFs_for_TG[idt])
            #print(TF_Select)
                
            TF_Select_Collect[k] = TF_Select


            MSE_[i] = Out_of_Sample_MSE(scRNA_Anndata, tg, TF_Select, SID_train, SID_outsample, pho)
            Map_Ind_k[i] = k
            #print(k, MSE_[i])
            
        # find the best k    
        best_k = Map_Ind_k[np.argmin(MSE_)]#Find_best_k(K_Range, MSE_)
        if best_k == -1:
            print("Fail to have good inference for ", tg)
        else:
            print(count, '/', len(TargetGenes), ":", tg, best_k, TF_Select_Collect[best_k], MSE_[K_Range.index(best_k)])
            GRIPNet_DF = GRIPNet_DF._append({'TG': tg, 'TFs': TF_Select_Collect[best_k], 'Out_of_Sample MSE': MSE_[K_Range.index(best_k)]}, ignore_index=True)
        count = count + 1
    return GRIPNet_DF
        
        

In [7]:
MSE = np.array([0.3, 0.2, 0.15, 0.4])

np.argmin(MSE)

2

# 1. Load data

## 1.1 Load scRNA-seq and trim to CD4

In [8]:
# load the data
Pbmc_3k_scRNA_DN_magic_ad = ad.read_h5ad("../../Data/PBMC_3k/pmbc_3k_scRNAseq_magic.h5ad")

In [9]:
Pbmc_3k_scRNA_DN_magic_ad.obs

,n_genes,doublet_score,predicted_doublet,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,ingest_celltype_label
AAACAGCCAAATATCC-1,2272,0.009280,False,2271,4746.0,369.0,7.774969,NK cells
AAACAGCCAGGAACTG-1,3254,0.020057,False,3253,7760.0,693.0,8.930412,CD14+ Monocytes
AAACAGCCAGGCTTCG-1,1798,0.037975,False,1793,3661.0,409.0,11.171811,CD14+ Monocytes
AAACCAACACCTGCTC-1,1145,0.016376,False,1142,2159.0,271.0,12.552108,B cells
AAACCAACAGATTCAT-1,1495,0.016376,False,1494,2909.0,293.0,10.072189,NK cells
...,...,...,...,...,...,...,...,...
TTTGTGGCATCCGTAA-1,1346,0.008130,False,1343,2560.0,432.0,16.875000,B cells
TTTGTGGCATTAGCCA-1,972,0.024421,False,969,1746.0,191.0,10.939290,CD4 T cells
TTTGTGGCATTGCGAC-1,2330,0.009280,False,2330,4772.0,316.0,6.621961,NK cells
TTTGTGTTCCGCCTAT-1,1057,0.054726,False,1056,1913.0,252.0,13.173027,CD4 T cells


In [10]:
# trim to CD4 
CD14_scRNA_DN_magic_ad = Pbmc_3k_scRNA_DN_magic_ad[Pbmc_3k_scRNA_DN_magic_ad.obs.ingest_celltype_label == 'CD14+ Monocytes'] 

In [11]:
# further filtering
CD14_scRNA_DN_magic_ad = CD14_scRNA_DN_magic_ad[:, CD14_scRNA_DN_magic_ad.var['mean']>=0.1]

#CD4_scRNA_DN_magic_ad = CD4_scRNA_DN_magic_ad[:, CD4_scRNA_DN_magic_ad.var['std']<=0.5]

In [12]:
Gene_in_CD14_scRNA = list(CD14_scRNA_DN_magic_ad.var.index)
Cell_in_CD14_scRNA = list(CD14_scRNA_DN_magic_ad.obs.index)

print(len(Gene_in_CD14_scRNA))
print(len(Cell_in_CD14_scRNA))

3055
702


## 1.2 Load scATAC-seq data and trim to CD14

In [13]:
pbmc_3k_scATAC_Gene_Enhancer = pd.read_pickle('../../Data/PBMC_3k/Pbmc_3k_scATAC_Gene_TF_Enhancer_fimo_multiple_cutoffs.pkl')

In [16]:
# Trim to Cells in CD8
Col_select = ['chr_position', 'Overlapped_enhancer',
 'Genes_within_250k',
 'TF_Matching_p-value_1e-06',
 'TF_Matching_p-value_1e-05',
 'TF_Matching_p-value_0.0001',
 'TF_Matching_q-value_0.01',
 'TF_Matching_q-value_0.05'] + Cell_in_CD14_scRNA
pbmc_3k_scATAC_Gene_Enhancer_CD14 = pbmc_3k_scATAC_Gene_Enhancer[Col_select]

In [17]:
# sum the peaks for cells in CD4
pbmc_3k_scATAC_Gene_Enhancer_CD14['sum'] = pbmc_3k_scATAC_Gene_Enhancer_CD14[Cell_in_CD14_scRNA].values.astype(int).sum(axis=1)

pbmc_3k_scATAC_Gene_Enhancer_CD14 = pbmc_3k_scATAC_Gene_Enhancer_CD14.drop(columns=Cell_in_CD14_scRNA)

/var/folders/x5/p97rd26d6y719tstlhqtn9zr0000gr/T/ipykernel_2416/1801729761.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbmc_3k_scATAC_Gene_Enhancer_CD14['sum'] = pbmc_3k_scATAC_Gene_Enhancer_CD14[Cell_in_CD14_scRNA].values.astype(int).sum(axis=1)


In [18]:
# consider only the open region with peaks >=100
peak_sum_threshold = 20
pbmc_3k_scATAC_Gene_Enhancer_CD14_psum = pbmc_3k_scATAC_Gene_Enhancer_CD14[pbmc_3k_scATAC_Gene_Enhancer_CD14['sum']>=peak_sum_threshold]

In [20]:
pbmc_3k_scATAC_Gene_Enhancer_CD14_psum

,chr_position,Overlapped_enhancer,Genes_within_250k,TF_Matching_p-value_1e-06,TF_Matching_p-value_1e-05,TF_Matching_p-value_0.0001,TF_Matching_q-value_0.01,TF_Matching_q-value_0.05,sum
4,chr1_267561_268455,"[3560:5.98727532125872:SUIT-2,1417:11.48107810...","[DDX11L17, FAM138A, LOC100132287, LOC112268260...","[RXRA, ZN770]","[ASCL1, COT1, COT2, CTCF, GATA3, GCR, HEN1, HE...","[AHR, ANDR, AP2A, AP2B, AP2C, AP2D, ARNT2, ASC...","[ZBT17, ZN341, ZN770]","[CTCF, PAX5, SP2, TAF1, ZBT17, ZF64A, ZFX, ZN1...",50
5,chr1_270864_271747,[.],"[DDX11L17, FAM138A, LOC100132287, LOC112268260...","[IRF3, PAX5, PRDM6, RREB1, SMAD3, ZFP28, ZN121...","[AP2C, AP2D, CEBPB, IRF1, IRF3, IRF5, MAFA, NF...","[ANDR, AP2A, AP2C, AP2D, ARI3A, ASCL1, ATF2, A...","[IRF3, PAX5, ZN121, ZN770]","[AP2D, CPEB1, IRF1, IRF3, PAX5, PRDM6, RARB, R...",28
8,chr1_629484_630393,"[550:12.2065412268318:SHEP-21N,787:6.466380502...","[FAM41C, FAM87B, LINC00115, LINC01128, LINC014...",[.],"[CDX2, GFI1, GFI1B, HAND1, HXC9, MYB, ONEC3, O...","[AIRE, ARI5B, BARX2, BATF3, BCL6, BSH, CDX2, C...",[.],"[RREB1, TAF1]",60
9,chr1_633556_634476,"[550:12.2065412268318:SHEP-21N,787:6.466380502...","[FAM41C, FAM87B, LINC00115, LINC01128, LINC014...",[NR2C1],"[GLI3, GSX2, HXB13, HXC9, INSM1, NR2C1, NRF1, ...","[BCL6, BMAL1, BSH, CDX2, CEBPZ, COT2, CUX1, DL...",[.],"[SP2, ZFX]",312
10,chr1_778284_779202,"[1980:8.91789239746368:CD8+,370:15.50080805540...","[AGRN, FAM41C, FAM87B, HES4, ISG15, KLHL17, LI...","[CEBPZ, FOXI1, NFYB, TEAD1, ZN563, ZN770]","[AP2D, ATF1, ATF2, ATF7, BHA15, CEBPZ, COE1, C...","[AIRE, AP2A, AP2B, AP2C, AP2D, ARNT, ASCL1, AS...",[ZN770],"[AP2D, E2F4, E2F6, EGR1, EGR2, KLF1, PROX1, PU...",884
...,...,...,...,...,...,...,...,...,...
98309,chrUn_GL000219v1_99257_100164,[.],"[LOC105379420, LOC283788]","[E2F1, E2F4, E2F6, E2F7, ELF1, ELF2, ELK1, ELK...","[AP2D, ARNT2, BHA15, E2F1, E2F3, E2F4, E2F6, E...","[ANDR, AP2A, AP2B, AP2C, AP2D, ARNT2, ASCL1, A...","[SP3, TBX15, WT1, ZN467, ZN770]","[AP2D, E2F4, E2F6, E2F7, EGR1, EGR2, EGR4, KLF...",1009
98312,chr11_KI270721v1_random_2089_2980,[.],[.],"[E2F4, EGR1, EGR2, EGR4, ESR1, ESR2, FLI1, KLF...","[AP2B, AP2D, ARNT2, ASCL2, BHA15, COT1, COT2, ...","[AHR, AP2A, AP2B, AP2C, AP2D, ARNT2, ASCL1, AS...","[EGR1, EGR2, FLI1, KLF1, KLF12, KLF15, KLF16, ...","[AP2D, E2F4, E2F6, E2F7, EGR1, EGR2, EGR4, ETV...",24
98316,chr1_KI270713v1_random_21434_22339,[.],"[LOC102724562, LOC124905321, LOC440570, TRN-GT...",[ZN341],"[COT1, CREM, E2F3, EGR4, EOMES, GLIS1, HEN1, H...","[AHR, ANDR, ATF2, BACH1, BACH2, BHA15, BHE41, ...",[ZN341],"[EGR4, KLF9, ZFX, ZN341]",1906
98317,chr1_KI270713v1_random_29578_30400,[.],"[LOC102724562, LOC124905321, LOC440570, TRN-GT...","[FOXG1, FOXL1, HEN1, OSR2, PAX5, PROX1, ZFP82,...","[ASCL1, ASCL2, BHA15, BMAL1, CDX2, FOXG1, FOXL...","[ANDR, ARNT2, ASCL1, ASCL2, ATOH1, BCL6B, BHA1...","[FOXL1, OSR2, PAX5, ZN121, ZN770]","[FOXL1, IRF3, KLF1, KLF5, KLF6, OSR2, PAX5, PR...",20


In [21]:
Gene_in_CD14_scATAC = DF_Column_to_List(pbmc_3k_scATAC_Gene_Enhancer_CD14_psum, 'Genes_within_250k')
print(len(Gene_in_CD14_scATAC))

Gene_in_CD14_scATAC_scRNA = list(set(Gene_in_CD14_scATAC) & set(Gene_in_CD14_scRNA))
print(len(Gene_in_CD14_scATAC_scRNA))

39716
2860


## 1.3 Load PPIs

In [22]:
G_PPI_BioGrid = pickle.load(open('../../Data/PPI/PPI_BioGrid_Evd2_DSD.pickle', 'rb'))
print("The PPI netowrk has", len(G_PPI_BioGrid.nodes), "nodes and", len(G_PPI_BioGrid.edges), "edges")

The PPI netowrk has 14433 nodes and 104148528 edges


# 2 GRIPNet

In [23]:
TF_Motif_Metric = ['TF_Matching_p-value_1e-06',
 'TF_Matching_p-value_1e-05',
 'TF_Matching_p-value_0.0001',
 'TF_Matching_q-value_0.01',
 'TF_Matching_q-value_0.05']

# split data into training and out-of-sample
scRNA_Anndata_Train, scRNA_Anndata_OutSample, SID_train, SID_outsample = Split_Sample_scRNA(CD14_scRNA_DN_magic_ad, percentage_training=0.9)

with open('./CD14_Train_Samples.txt', 'w') as f:
    for sid in SID_train:
        f.write(f"{sid}\n")
        
with open('./CD14_Out_Samples.txt', 'w') as f:
    for sid in SID_outsample:
        f.write(f"{sid}\n")


In [27]:
TGList = ['BCL2A1', 'CBR4', 'TCF7', 'SLC24A4', 'FAM241A', 'TMEM117', 'TMEM154', 'SH2D3C', 'RASGRF2', 'CRISPLD2', 'BCL11A', 'ELP4', 'DPYD', 'DHRS7B', 'PLXNC1', 'KCNE1', 'SNTB1', 'HIST1H1D', 'S100A12', 'RFFL', 'NCALD', 'MAP3K14', 'GNB4', 'PTPN22', 'MXD1', 'IFT74', 'SH3RF1', 'SYK', 'PFKFB4', 'NUP50-DT', 'PRKACB', 'ITGA6', 'IFT88', 'EIF4ENIF1', 'LILRB1', 'LILRB2', 'RELL1', 'AHCYL2', 'SOCS3', 'CPVL', 'TBC1D8', 'SHLD2', 'SFMBT1', 'TDRD3', 'STYXL1', 'PITPNC1', 'LRSAM1', 'MCTP1', 'ZNF830', 'GIT2', 'MSRA', 'CEP290', 'FOSB', 'PRR5', 'CD244', 'SGK3', 'CD22', 'NSMAF', 'LARP1B', 'DEPDC5', 'HDAC8', 'BANK1', 'MYC', 'ITFG1', 'CD2AP', 'FAM135A', 'IMP3', 'TOM1', 'UBA7', 'CSTA', 'KMT2B', 'CCSER2', 'APBA2', 'KAT6B', 'NUBPL', 'SLC25A37', 'VTA1', 'CD3E', 'TACC1', 'JMJD1C', 'CLEC4E', 'BASP1', 'SNX30', 'PARP14', 'ZBTB16', 'CNKSR2', 'TLR4', 'AKAP10', 'NIPSNAP2', 'TIPARP', 'CREB5', 'S100A8', 'LTB', 'INPP4B', 'S100A10', 'WDFY1', 'CEP78', 'LRRFIP2', 'MINDY2', 'RPS27', 'NARS2', 'LMNA', 'SMYD3', 'ATP1B3', 'GALNT3', 'ZMAT3', 'DOP1B', 'SSR4', 'HIST1H1E', 'PRAM1', 'TAPT1-AS1', 'CHPT1', 'INTS1', 'CD6', 'LRRC8B', 'EHBP1', 'PIM2', 'RBL2', 'TPT1', 'TBC1D19', 'ARHGAP26', 'TECPR1', 'LY9', 'TUBD1', 'GPR137B', 'ZC4H2', 'FAM102B', 'TNFRSF10B', 'RORA-AS1', 'AOAH', 'TBC1D20', 'TPCN1', 'MAN1C1', 'CYSLTR1', 'UBE2D2', 'UBE2E2', 'ZNF75D', 'SMIM8', 'HMGB2', 'TENT5A', 'CARMIL1', 'GZMH', 'TERF2IP', 'EPHA4', 'MECP2', 'SNX2', 'MYOF', 'ABHD5', 'FCHSD2', 'CTSW', 'NFIA', 'CYFIP1', 'PCNX4', 'NOL4L', 'ZNF81', 'FBXO33', 'NDUFAF6', 'RORA', 'PHTF1', 'STAMBPL1', 'ENSA', 'PTPN12', 'LDHB', 'BMP2K', 'CCND2', 'ZDHHC17', 'FBXL20', 'IFITM3', 'S100A11', 'SLC7A6', 'C12orf42', 'GAS7', 'MAPKAPK5-AS1', 'RNF115', 'ARRB1', 'OLA1', 'CR1', 'PLD4', 'OSBPL1A', 'CLIC4', 'LRRK1', 'RPS29', 'WDR60', 'ME2', 'LACTB', 'FMN1', 'USP40', 'ATG10', 'PELI2', 'NLRP3', 'GNG7', 'IGF2BP2', 'MNDA', 'MEF2C', 'MS4A7', 'RETREG1', 'ZNF814', 'PHF21A', 'RAB35', 'EHMT2', 'MEGF9', 'KLRG1', 'SUSD3', 'KCNQ1OT1', 'ERI1', 'RAB20', 'HERC5', 'TENT5C', 'USP6NL', 'DGKE', 'PUDP', 'RALGAPA2', 'PAM', 'MCTP2', 'LCK', 'HACD3', 'PTPN4', 'SYAP1', 'SYNJ2', 'CDA', 'DLEU2', 'INTS4', 'ANKRD28', 'FRMD4B', 'CTNNBIP1', 'NFATC2', 'MYADM', 'PRAG1', 'MT2A', 'ADCY9', 'DAAM1', 'PKP4', 'DAPK1', 'NEDD9', 'CDIPT', 'PACSIN2', 'KIAA2013', 'ACTR1B', 'PRR14L', 'NSUN6', 'MOSMO', 'LINC00513', 'PIGX', 'SLC2A13', 'AFF3', 'EAF2', 'YARS', 'SNTB2', 'C19orf53', 'CROCC', 'SEC22A', 'ABCC4', 'PPP1R12B', 'FCAR', 'RABGEF1', 'EEF1B2', 'PELI1', 'INKA2', 'ADAP2', 'ERCC8', 'CENPK', 'TMCC1', 'SCML1', 'ZFHX3', 'SMARCD1', 'ST6GALNAC3', 'NDUFAF2', 'TBC1D32', 'PITPNA', 'UVRAG', 'MRPS9', 'C12orf57', 'ZNF827', 'SCARB2', 'ARMC1', 'TRIM52', 'ATP10A', 'CABIN1', 'XPNPEP3', 'AP3S2', 'ARID5B', 'JDP2', 'FER', 'SASH1', 'PDE7B', 'NOD2', 'ANKZF1', 'DNAJC1', 'ABCD2', 'P4HA1', 'RPS5', 'ABL2', 'CMIP', 'TTN', 'MAP2K1', 'RPL13', 'TPK1', 'OFD1', 'EMILIN2', 'UBE2E1', 'IPMK', 'SLC10A7', 'TUT4', 'IL2RB', 'TCAIM', 'FXN', 'FRMD3', 'HERC6', 'IFT57', 'NCF2', 'SNED1', 'C11orf49', 'ST3GAL3', 'MS4A4E', 'NABP1', 'DGKH', 'DGKA', 'TMEM161B-AS1', 'ZNF611', 'SUSD1', 'TSPAN5', 'CASK', 'BIRC3', 'ZNF718', 'TRPS1', 'MARK4', 'LINC01237', 'DHTKD1', 'SLFN11', 'MATK', 'ARSG', 'MSH3', 'PIP5K1B', 'HS3ST3B1', 'DHX34', 'CPPED1', 'ALDH2', 'DST', 'TAMM41', 'ECHS1', 'RAP1GDS1', 'RASA3', 'SLC25A13', 'RBM47', 'NIPAL2', 'RRBP1', 'BTD', 'ING5', 'WDR25', 'EPSTI1', 'APC', 'DIPK2A', 'PIGN', 'EVI5', 'LPCAT2', 'KLHDC10', 'NPL', 'ARHGEF40', 'PRKD2', 'SETD7', 'TESK2', 'SETBP1', 'STAT5B', 'HIP1', 'CD79A', 'SHISAL2A', 'ADARB1', 'MAP4', 'RBL1', 'ODC1', 'PLCL1', 'ARHGAP5', 'KRI1', 'SIK2', 'LINC01184', 'MTERF4', 'FAM122C', 'KIF1B', 'ID2', 'CAAP1', 'CD5', 'UBASH3A', 'LINC00662', 'TLR2', 'RAE1', 'EPHX2', 'CLOCK', 'RASSF1', 'CRTC1', 'DMXL2', 'STAM', 'RPS27A', 'ARRDC3', 'ZNF438', 'BLOC1S4', 'RPL23A', 'RNF19B', 'PDK3', 'CTNND1', 'BMT2', 'RFX7', 'PDLIM5', 'RPS3', 'ZFP90', 'VPS35L', 'ZNF490', 'SLC31A2', 'OAS3', 'LDAH', 'TIMM8B', 'MXI1', 'PID1', 'MAP4K3', 'SCIMP', 'SPATA1', 'SAMSN1', 'ITM2A', 'SAMD3', 'SLC9A9', 'KDM7A', 'UBE2E3', 'RPL14', 'HSDL2', 'CCR2', 'AHNAK', 'KANTR', 'DENND1A', 'PHLPP1', 'CTC1', 'DHX57', 'NBEAL2', 'ITGB3BP', 'EXOSC8', 'COTL1', 'DUSP5', 'CLYBL', 'PIBF1', 'ZC3H6', 'ODF2L', 'OSCAR', 'NHSL1', 'LANCL2', 'AGPAT5', 'LDLRAP1', 'RTTN', 'HSPA1A', 'TCP11L1', 'TMTC2', 'SIPA1L3', 'SLC16A1-AS1', 'CYFIP2', 'PIK3AP1', 'IMMP1L', 'ACVR2A', 'EEF1A1', 'PADI4', 'SESN3', 'TTYH3', 'SNX4', 'NEAT1', 'ZDHHC14', 'UBL3', 'PRKCH', 'EIF4G3', 'GBP5', 'MMS22L', 'CYB561A3', 'RBM41', 'PACS2', 'SLC25A16', 'ATG2A', 'PTAFR', 'SNX29', 'LRMDA', 'FBXO22', 'JUN', 'ARHGAP21', 'GYPC', 'PPM1L', 'CD79B', 'HNRNPLL', 'ZNF804A', 'UBE2D1', 'LCORL', 'TUBA1A', 'NBEAL1', 'HIST1H4C', 'CCNC', 'GPD2', 'IFI30', 'PHACTR1', 'USP37', 'PLCG1', 'SIDT1', 'LINC02245', 'FAAH2', 'BRWD3', 'FBN2', 'AK5', 'VEGFB', 'NUAK2', 'DUSP16', 'VAV2', 'FKBP11', 'KLRD1', 'ABHD3', 'TNFAIP2', 'NFAT5', 'NR4A3', 'TATDN3', 'MRPS26', 'SLC25A25', 'LBH', 'TSTD1', 'SPTLC2', 'PLAGL1', 'SCLT1', 'LIME1', 'SPI1', 'MAPK8', 'CFD', 'PLEKHA3', 'C11orf80', 'GBP1', 'RCAN3', 'ETV6', 'ZNF654', 'CD83', 'HDAC3', 'CDC14A', 'TOR1B', 'CHSY1', 'SRGN', 'STX8', 'C1orf56', 'NOL11', 'ING3', 'ATP2B1', 'AK9', 'CTDP1', 'RRP12', 'HIVEP2', 'TREM1', 'UGGT2', 'TRAPPC8', 'CLASRP', 'MOB3B', 'DENND3', 'STMN3', 'ALPK1', 'NUMB', 'SAMD4A', 'RPS3A', 'NKG7', 'PPP3CA', 'GFM2', 'TRG-AS1', 'GLCCI1', 'ACADM', 'NAA60', 'OPA3', 'LEF1-AS1', 'CCDC141', 'GOLGA8A', 'DISC1', 'PPP2R2B', 'ENOX2', 'ACSL4', 'PLXDC2', 'RIN3', 'LRRC37A3', 'SLF1', 'DZIP3', 'PILRA', 'RIN2', 'DENND2D', 'ALMS1', 'SNRNP27', 'TTYH2', 'CC2D2A', 'SMURF1', 'RAB11FIP4', 'CASC4', 'IGF1R', 'DLG2', 'RGS18', 'CST7', 'PIGB', 'TIAM1', 'TDP2', 'LILRB3', 'FCMR', 'POP5', 'ATG7', 'IRS2', 'HCK', 'MYO9A', 'HIVEP1', 'GLUL', 'XAF1', 'RAP1A', 'LRRC8D', 'LINC01550', 'QSER1', 'CTSC', 'VCPKMT', 'EMSY', 'ATP6V0A1', 'PHACTR4', 'PSIP1', 'NR3C2', 'DDX58', 'IDH3A', 'RNF125', 'PRIM2', 'C20orf194', 'OPTN', 'SNAP29', 'XBP1', 'AQP3', 'MCCC1', 'ARHGEF11', 'PDK1', 'NRIP1', 'ZBP1', 'SUFU', 'CCDC167', 'ZNF821', 'PCSK5', 'RECK', 'HRH2', 'TSC22D1', 'CHURC1', 'CCDC88A', 'FCGR1A', 'RFX2', 'GALK2', 'PDE5A', 'FRS2', 'RCN2', 'BMPR2', 'TMEM120B', 'TBC1D9', 'TVP23C', 'SPIN1', 'PCCA', 'ZSWIM8', 'CHST15', 'UVSSA', 'FAR2', 'PDE8A', 'MICAL2', 'LINC00243', 'GASK1B', 'CD69', 'DDX60', 'GANC', 'IL1B', 'AHI1', 'RPL18A', 'CEACAM21', 'RAPGEF6', 'S100Z', 'SHQ1', 'NCK2', 'PATL2', 'PDE3B', 'HLA-DQA1', 'UBE3D', 'CCSER1', 'CXCR4', 'KIFAP3', 'C19orf12', 'RPL13A', 'PRKCQ', 'TPD52', 'RUBCNL', 'WARS', 'CALR', 'ACTG1', 'OSBPL10', 'TRAPPC13', 'FGFBP2', 'CEP250', 'AIF1', 'UBE2W', 'PRF1', 'SLC11A1', 'COG2', 'MBNL3', 'NIBAN3', 'FAM168A', 'IRF1', 'ACVR1', 'CDK19', 'PRDM1', 'ATRN', 'MAP3K8', 'ITPKB', 'ST8SIA4', 'ELP2', 'PPARGC1B', 'IFITM1', 'RPS20', 'MTMR3', 'RHBDD1', 'BACH2', 'PRMT3', 'NOLC1', 'ZC2HC1A', 'DPH5', 'EEPD1', 'ANXA1', 'DUSP2', 'USP28', 'TRAF3', 'TMOD2', 'HDAC9', 'TRIM25', 'GK5', 'CCDC50', 'TRAK1', 'CARNMT1', 'KIF16B', 'RHOC', 'IFI6', 'ERLEC1', 'PLPP1', 'TASOR', 'RASGRP1', 'RPS12', 'SLC49A4', 'CD38', 'UBN1', 'RPGR', 'GIMAP5', 'UMAD1', 'FCN1', 'HK2', 'EPB41L2', 'INSR', 'PSMA1', 'SRGAP2', 'RPL30', 'FAM20B', 'CCDC18', 'MX2', 'KCNQ1', 'QSOX1', 'TMED8', 'ACACA', 'TRPC4AP', 'GPAT3', 'PTK2', 'MYO9B', 'ARL13B', 'RPS18', 'METTL15', 'CTSZ', 'APOBEC3G', 'DNAJC24', 'MAP3K20', 'PECAM1', 'RPL31', 'TEX10', 'MPEG1', 'PHTF2', 'PKD2', 'LCLAT1', 'CUTC', 'FUT8', 'ARHGEF10L', 'GBP4', 'CUX1', 'SLC9A7', 'RNF130', 'LY86', 'ARHGAP31', 'PPM1K', 'IRF7', 'RRM1', 'SEPTIN1', 'KLF12', 'WDR49', 'CLUAP1', 'USP53', 'JAK2', 'ANKH', 'HERC3', 'OSBPL3', 'KMO', 'ARL4C', 'OSBPL5', 'CTSS', 'CCDC7', 'CCND3', 'NR4A1', 'SMIM14', 'MX1', 'RNASE6', 'XKR6', 'MYO1F', 'B4GALT5', 'ITGAM', 'HMGXB4', 'GPATCH2', 'ZNF331', 'NBEA', 'RIPK2', 'MCM5', 'GPANK1', 'PMAIP1', 'STX6', 'GLMN', 'MIR181A1HG', 'SIPA1L1', 'MARCKS', 'TSC22D2', 'NCF1', 'REEP3', 'GTDC1', 'PLSCR1', 'ISG15', 'ARFGAP3', 'PRKCE', 'PAK1', 'BCL2', 'MRTFB', 'UXS1', 'DTWD1', 'DYNLL1', 'INPP5F', 'PRKCA', 'ANKRD22', 'LRRC25', 'GRK3', 'NFKBIA', 'HIVEP3', 'LIMS1', 'SNAPC5', 'TAPT1', 'BCR', 'ICOS', 'PDSS2', 'RPL11', 'JARID2', 'FGD2', 'NMT2', 'RGS10', 'GNL2', 'MPZL3', 'ANKAR', 'HSPA4', 'AFG1L', 'SESTD1', 'AMN1', 'UGCG', 'TFDP2', 'NF1', 'ACTN1', 'CD82', 'PLCB1', 'RPL9', 'LINC00894', 'HIBCH', 'NDUFV2-AS1', 'NOTCH2', 'CASTOR3', 'EIF2B3', 'ERCC3', 'WARS2-AS1', 'SIRPB2', 'LINC02446', 'THEMIS', 'POC1B', 'MSI2', 'DENND11', 'GNS', 'PRR5L', 'SATB1', 'TTC27', 'LGALS2', 'NLK', 'MARCH3', 'SLC38A1', 'CPED1', 'GXYLT1', 'ANKRD36C', 'SLC22A15', 'ATP2B4', 'NFAM1', 'GTPBP6', 'APOBEC3A', 'FAM153CP', 'IDH1', 'CHMP7', 'MAML2', 'LPCAT1', 'NAMPT', 'SUPT3H', 'RPAP2', 'GPR183', 'RRAS2', 'TAFA2', 'ZNF544', 'GBF1', 'TNS3', 'S100A6', 'LGALS1', 'RALGPS2', 'OAS1', 'PDZD8', 'PWWP3A', 'SOS2', 'HLA-DMB', 'ARMH1', 'ITPK1', 'CD68', 'ANPEP', 'MVB12B', 'TMEM164', 'MEF2C-AS1', 'IL13RA1', 'TAF4B', 'MCM9', 'TNRC6C', 'CIAPIN1', 'C9orf85', 'PDE4D', 'FKTN', 'DROSHA', 'CD2', 'LRRC28', 'IFIH1', 'MAP2K6', 'OXR1', 'SAMD12', 'EXT1', 'ZCCHC10', 'EFCAB2', 'CARD11', 'TNFAIP3', 'SLAMF1', 'MANF', 'ZSWIM6', 'CENPP', 'ABLIM1', 'PACS1', 'TESPA1', 'PTGER2', 'C5AR1', 'MICU1', 'CSGALNACT1', 'TYROBP', 'NAA35', 'TBXAS1', 'ARHGAP15', 'RAD17', 'AKT3', 'AP3M2', 'MTSS1', 'PPFIBP2', 'PLEKHA2', 'VRK1', 'TFCP2', 'PCNX2', 'LMTK2', 'AIG1', 'USF3', 'VIM-AS1', 'ADAM9', 'JAZF1', 'KLHL5', 'BTBD11', 'FAM222B', 'FOXO3', 'HLA-DMA', 'DENND1C', 'BHLHE40', 'WRN', 'ACSL1', 'TTC39C', 'SOD2', 'AGAP3', 'CTBP2', 'ZNF33B', 'SIMC1', 'TIGAR', 'NPC1', 'CBX5', 'SYTL2', 'EGLN3', 'PLD1', 'CCDC124', 'SLC35E2B', 'TCF7L2', 'EPB41L3', 'RNF149', 'FBXW8', 'HEG1', 'CDC42EP3', 'SLC44A1', 'ITGAX', 'TUBGCP5', 'CFP', 'FNTB', 'ITPR1', 'FCER1G', 'NIPAL3', 'C1orf52', 'EFL1', 'SERPINB9', 'MAP3K5', 'BSDC1', 'RNF144A', 'TFRC', 'ZNF169', 'CLEC7A', 'PALM2-AKAP2', 'DGLUCY', 'MPRIP', 'SLC44A2', 'SHLD1', 'ZDHHC13', 'DNAJB1', 'ANTXR2', 'RAB27A', 'LINC00926', 'IL32', 'RPS6KA4', 'HIST1H2AC', 'SFXN5', 'CLIP4', 'TMEM176B', 'CDC25B', 'RNF144B', 'TRMT11', 'LIX1L', 'HS2ST1', 'SBF2', 'ZBTB38', 'KDM5B', 'TTF1', 'CBLB', 'PDE6D', 'ATXN3', 'EIF2AK3', 'GBE1', 'TMEM223', 'MAF', 'TEC', 'DHRS3', 'SCML4', 'ATF1', 'CSF3R', 'INVS', 'GCFC2', 'SEPTIN11', 'GAB2', 'DLEU1', 'BLK', 'SLC16A10', 'VNN2', 'GPRIN3', 'DRG1', 'AP4E1', 'TNFSF10', 'DCPS', 'RXRA', 'FOCAD', 'PHLDB3', 'BBS9', 'GPR171', 'TEX14', 'PITPNM2', 'PCED1B', 'HCFC2', 'STX7', 'TFEC', 'COMMD10', 'STK3', 'CD8B', 'NFIL3', 'ATG4C', 'GRN', 'MFAP1', 'EP400', 'RAB28', 'UBXN2A', 'APOLD1', 'POLD3', 'POLA1', 'CREB3L2', 'ABI2', 'VCAN', 'ABR', 'PTPRM', 'BICDL1', 'PASK', 'GRAMD1B', 'SERINC5', 'YBX3', 'HIPK2', 'SPECC1', 'CYBB', 'UBAP1', 'ATP8B1', 'POLR3C', 'ADAM28', 'HCST', 'LYZ', 'SLC4A7', 'SCLY', 'RPS4X', 'ATF3', 'ALKBH7', 'LAIR2', 'IL1RAP', 'SLC43A2', 'CAMK2D', 'LLGL2', 'PLAAT4', 'LINC00877', 'GCNT1', 'RAB6A', 'STMN1', 'TMX4', 'CCL5', 'ATP6AP2', 'DOCK5', 'FCER2', 'LINC02328', 'FAM117B', 'VPS54', 'DPP4', 'FGL2', 'KIF13A', 'SCO2', 'HMGCS1', 'IL4R', 'CIITA', 'C12orf75', 'BCAS4', 'VMP1', 'TTBK2', 'KYNU', 'ACAD11', 'EVA1C', 'ARSB', 'CCDC200', 'FBXL5', 'BTRC', 'CEP162', 'SPPL2A', 'NOSIP', 'SGMS2', 'SLC31A1', 'SARNP', 'RGS2', 'HLA-DRB5', 'TMC8', 'UTRN', 'TPGS2', 'RMND1', 'DIAPH2', 'ASAP1', 'LIMD1', 'STK17A', 'CD3D', 'RAB31', 'CASS4', 'TBC1D2', 'CYTOR', 'TRAF3IP2-AS1', 'RNF135', 'UBTF', 'CDKN1C', 'RUNX2', 'SDCCAG8', 'UST', 'DCAF10', 'MTG1', 'SRRT', 'LEF1', 'GZMB', 'WDFY3', 'IRAK3', 'BATF', 'AHR', 'SLC16A7', 'HIBADH', 'COG6', 'WHAMM', 'ISCA1', 'DDB2', 'EPS8', 'CPLANE1', 'ATF7IP2', 'UBASH3B', 'ATP11A', 'CHKA', 'MANBA', 'SPATS2L', 'UBE2J1', 'DHX35', 'CMSS1', 'CALCOCO1', 'TSEN2', 'DUSP1', 'ALOX5AP', 'VRK2', 'NSD2', 'ATAD2', 'WDR27', 'ACYP2', 'MS4A1', 'SLC35E3', 'HIF1A', 'SYNE2', 'PFN1', 'WWOX', 'TSHZ3', 'TNFRSF1B', 'LINC01572', 'ELL2', 'POU2F2', 'DPH6', 'ZNF251', 'CPNE8', 'HIST1H1C', 'BAZ2B', 'ZNF236', 'CEBPB', 'RNF168', 'MOB1B', 'RPLP1', 'SH3RF3', 'TOGARAM1', 'GPHN', 'SLAMF6', 'ZEB2', 'LINC00861', 'NLN', 'ABCB10', 'FANCC', 'CD14', 'PTPN7', 'KSR1', 'ITPRIP', 'HIPK3', 'DTNB', 'PPP1R3E', 'NLRC3', 'GLT1D1', 'MCU', 'ZRANB3', 'SERPINA1', 'TNIK', 'SLMAP', 'TMCO4', 'CD86', 'HABP4', 'CD8A', 'XRCC4', 'HLA-DRB1', 'AOPEP', 'LATS2', 'SPIDR', 'RYBP', 'PRMT9', 'EXOC6', 'WDR6', 'METTL8', 'OXSR1', 'TYW5', 'TECPR2', 'SLC8A1', 'RNF6', 'ETS1', 'USP32', 'SNX9', 'CD96', 'RPS6', 'CCAR2', 'TMEM156', 'MS4A6A', 'ABCG1', 'ITK', 'TECR', 'GRAP2', 'ESR1', 'PDE4B', 'IFIT2', 'HLA-DQB1', 'ARHGAP24', 'SMARCAL1', 'STX11', 'RSAD2', 'ZDHHC6', 'PRH1', 'CMTM8', 'GNAQ', 'FTL', 'CLIC3', 'DUSP10', 'BCKDHB', 'EXOC6B', 'PCGF5', 'OSBPL2', 'IFI44L', 'KATNBL1', 'VOPP1', 'SVIL', 'SEMA4B', 'TTLL5', 'CDK6', 'S100A9', 'IDE', 'CPD', 'SIL1', 'CALU', 'FLVCR2', 'HAVCR2', 'SLC35D1', 'MEAF6', 'FLNB', 'CD36', 'ZAP70', 'ATP8A1', 'CNNM2', 'MAPKAPK2', 'PSTPIP2', 'PEAK1', 'CD28', 'SLC4A8', 'ITGB1', 'ARRDC2', 'PRKAG2', 'CD7', 'LANCL1', 'PYHIN1', 'TMUB1', 'MTAP', 'ABHD12', 'ZNF710', 'ZNF433-AS1', 'TANGO6', 'MIS18BP1', 'NGRN', 'TRERF1', 'ATG13', 'CDK14', 'RPL35A', 'RHOQ', 'NUCB2', 'B4GALT1', 'TGFBR3', 'SYNJ1', 'TBC1D14', 'NUP54', 'MPP7', 'LINC01278', 'EXOSC3', 'DOCK7', 'ZFYVE28', 'ABCA5', 'SRBD1', 'PPCDC', 'L3MBTL4', 'INTS12', 'GALNT10', 'ATP8B2', 'TMEM192', 'MYO1E', 'ITSN1', 'CHST11', 'SKAP1', 'DGKG', 'MFHAS1', 'DNAJC17', 'IFITM2', 'AMZ1', 'SFXN1', 'SMC6', 'PLA2G4A', 'MPP6', 'LRIG1', 'PARP15', 'EXT2', 'FCRL1', 'EFHD2', 'TSHZ2', 'WDPCP', 'AUTS2', 'VKORC1L1', 'ARHGEF18', 'ZNF43', 'SCMH1', 'CDK8', 'PTPRJ', 'DOCK9', 'CXXC5', 'MAST2', 'MYBL1', 'LPAR1', 'RASGEF1B', 'ANKS1A', 'CRYZL1', 'CEP295', 'ST3GAL6', 'TRIO', 'HLA-DPA1', 'GALT', 'SFMBT2', 'CDH23', 'GCNT2', 'KDM1B', 'HPS5', 'ENTPD1-AS1', 'ZNF487', 'LYN', 'NT5C2', 'MARCKSL1', 'PBX3', 'CD81', 'LEPROTL1', 'SYNE1', 'NFKBIZ', 'CPEB4', 'CBFA2T2', 'TRAF1', 'LDLR', 'GNB5', 'CD3G', 'ATG16L1', 'SOX4', 'ADGRE5', 'CKMT2-AS1', 'HNMT', 'CMC1', 'RILPL2', 'PLEKHM3', 'MAGED2', 'MCPH1', 'SIK3', 'TET3', 'TNFRSF25', 'YPEL2', 'RPS6KC1', 'STAT4', 'RMND5B', 'CAMK4', 'SEC24A', 'TPMT', 'UFC1', 'ICAM1', 'C2CD3', 'CD226', 'ABTB1', 'NEGR1', 'VCL', 'KANSL1L', 'MTX2', 'DENND5A', 'STRBP', 'MRPL1', 'TMEM170B', 'BACH1', 'NEK6', 'ORC3', 'TRMO', 'SGPP1', 'CTSD', 'MEF2A', 'PTPRE', 'PIGL', 'ZMYND11', 'FRY', 'CCDC66', 'SPTBN1', 'INPP5A', 'PYM1', 'WDFY2', 'PIK3IP1', 'SEL1L3', 'SGK1', 'MAL', 'DDX60L', 'APLP2', 'RPL27A', 'SLC25A33', 'TRABD2A', 'NME7', 'PPARA', 'RPL34', 'PPM1D', 'SLC39A11', 'NDFIP1', 'NIBAN1', 'ANKRD26', 'STK25', 'HVCN1', 'RELB', 'HOPX', 'POLR1C', 'KIAA1958', 'SPATA5', 'KRCC1', 'IFIT3', 'MGST2', 'EPB41', 'ENTPD1', 'TMEM135', 'ITGB7', 'FBXO32', 'TMEM267', 'E2F3', 'FOS', 'NELL2', 'SLAMF7', 'RIC8B', 'ECHDC2', 'WWP2', 'ANKIB1', 'RHOH', 'LYST', 'IPO11', 'MAST4', 'TCF4', 'RPS2', 'KHDRBS2', 'SMAD3', 'NLRP12', 'LIN52', 'LARS2', 'DHPS', 'FNDC3B', 'B2M', 'SMS', 'SCFD2', 'PLEK', 'LINC00937', 'SULF2', 'RPL38', 'MAML3', 'SWAP70', 'CCDC146', 'SAT1', 'CD55', 'ORAI2', 'RPS6KA2', 'ABL1', 'RHOT1', 'SKAP2', 'RNF24', 'ANK3', 'PSAP', 'PYGL', 'MED31', 'HLCS', 'LRRK2', 'SLC38A9', 'ZNF10', 'PRKN', 'IL7R', 'FARP2', 'ARHGEF3', 'LTBP3', 'RPL19', 'RPS26', 'MRPS18B', 'TC2N', 'PIGU', 'RSU1', 'BCL6', 'NCEH1', 'GRIK1', 'DUSP6', 'GRK5', 'ACTB', 'LRBA', 'ERGIC1', 'C9orf72', 'SLC25A12', 'CD247', 'TMEM144', 'FOXO1', 'KCNQ5', 'CSF2RA', 'KIAA0355', 'TXK', 'KIF20B', 'SLC17A5', 'TBC1D4', 'ZNF565', 'GAB1', 'COMMD1', 'ADGRE2', 'RPL32', 'GALM', 'KIAA0556', 'RAPGEF2', 'FAM117A', 'OXCT1', 'NCAPD2', 'AAMDC', 'CASC3', 'PLAUR', 'SNX25', 'ATP8B4', 'BCAS3', 'RTN1', 'FCGR2A', 'FGR', 'LIPE-AS1', 'RPS15A', 'CCR7', 'KLHL8', 'TET2', 'CLEC12A', 'ZNF37A', 'MAGED1', 'CLMN', 'LAT', 'GATA3', 'LAP3', 'OASL', 'ABAT', 'PICALM', 'NR4A2', 'RAB11FIP1', 'DYSF', 'STXBP5', 'RPSA', 'IRF8', 'NAE1', 'CPEB3', 'ZFAT', 'RNF157', 'TXNDC16', 'DOCK4', 'NCBP2AS2', 'PLEKHG2', 'LRP1', 'GABPB1-AS1', 'IFT80', 'S100A4', 'TRIP4', 'ACER3', 'SIRPG', 'ACSS2', 'SELL', 'IL21R', 'SLX4IP', 'DNAJA1', 'ZNF782', 'MFSD11', 'FBXO34', 'CYSTM1', 'ADAM8', 'ESR2', 'TASP1', 'FMNL2', 'ARHGAP10', 'GPCPD1', 'DSTYK', 'LYPLAL1', 'NAA10', 'CSNK1G3', 'FAM49A', 'KLF4', 'CST3', 'EZH2', 'ANKRD36B', 'DOCK8', 'SNX10', 'CISH', 'ATXN7L1', 'SPAG9', 'SLC2A3', 'CHRM3-AS2', 'MAN1A1', 'THAP11', 'SLCO3A1', 'ARAF', 'AGO4', 'COL19A1', 'BABAM2', 'MIR646HG', 'CD27', 'CSTB', 'PTPRK', 'YJU2', 'TAFA1', 'TTC7A', 'RPL3', 'OCIAD2', 'LINC00649', 'FASTKD1', 'CHD7', 'ARHGAP32', 'CDR2', 'RTN3', 'PVT1', 'HSPB1', 'BICD1', 'AGPAT4', 'SRGAP2B', 'SLC39A9', 'RPL10', 'MIB1', 'ORC5', 'MRPL28', 'WARS2', 'TOX', 'SRD5A1', 'FGGY', 'FILIP1L', 'PCNA', 'IKZF3', 'CRTC3', 'WDFY4', 'ZNF512', 'GFPT1', 'ZEB1', 'CCAR1', 'LENG1', 'APP', 'CEBPD', 'CD74', 'IL15', 'MICAL3', 'FHIT', 'LINC02432', 'GMEB1', 'AGTPBP1', 'PLAC8', 'ABCA7', 'GZMA', 'GPATCH11', 'ATXN1', 'THEM4', 'TMEM263', 'XYLT1', 'AKR1B1', 'OXNAD1', 'SCAI', 'MLLT3', 'KIAA0825', 'CARS2', 'ZNF831', 'COX10-AS1', 'IFRD1', 'BCL11B', 'HLA-DRA', 'CSF1R', 'PPP1R15B', 'TMEM260', 'IL17RA', 'RABEP2', 'ZNF277', 'PLBD1', 'RPS14', 'COLGALT1', 'B3GNTL1', 'LUZP1', 'ZDHHC20', 'LYRM7', 'PAX5', 'SPAST', 'TYMP', 'LRRN3', 'ENOSF1', 'EMD', 'GMDS-DT', 'ANO10', 'MIR4435-2HG', 'RGCC', 'PDE7A', 'TYW1B', 'LIMK2', 'CPQ', 'IL15RA', 'HLA-DPB1', 'GCH1', 'USP36', 'SLC2A4RG', 'C6orf89', 'FAM198B-AS1', 'MPP5', 'BCAS2', 'FBXO28', 'CPB2-AS1', 'LINC01934', 'SLC15A4', 'CRADD', 'CYHR1', 'SLFN12L', 'CYB5D2', 'SATB1-AS1', 'ADAM19', 'SPOCK2', 'SFI1', 'RAB3IP', 'GZMM', 'PPP1R16B', 'GK', 'P2RX7', 'IFI44', 'PTCD3', 'PNPT1', 'CCPG1', 'FAM13A', 'DIP2A', 'ARHGAP18', 'P2RX1', 'RPS25', 'IQCB1', 'NEK1', 'ALOX5', 'QKI', 'FGD4', 'HEATR5A', 'ANKUB1', 'ALCAM', 'EBF1', 'PATJ', 'GABPB1', 'RPS23', 'DIPK1A', 'MAFB', 'LST1', 'ERN1', 'GPATCH2L', 'HSH2D', 'ZBTB8OS', 'ZNF516', 'ULK4', 'SMIM4', 'NEPRO', 'ZNF276', 'IL3RA', 'MGAT4A', 'TRPM2', 'TRAT1', 'KLRK1']

GRIPNet_Res = Run_GRIPNet(CD14_scRNA_DN_magic_ad, SID_train, SID_outsample, pbmc_3k_scATAC_Gene_Enhancer_CD14_psum, G_PPI_BioGrid, TF_Motif_Metric[2], TargetGenes = TGList[800:], Lambda = 0.001)

1 / 1046 : XKR6 9 ['MYC', 'ESR1', 'JUN', 'SMAD3', 'ZEB1', 'MECP2', 'RXRA', 'ESR2', 'FOS'] 0.004188702974443397
2 / 1046 : MYO1F 10 ['MYC', 'ESR1', 'HIF1A', 'JUN', 'SMAD3', 'ESR2', 'MECP2', 'CEBPB', 'ZEB1', 'RXRA'] 0.007562290429236701
3 / 1046 : B4GALT5 10 ['GATA3', 'RORA', 'ZEB1', 'FOS', 'MYC', 'JUN', 'ESR1', 'SMAD3', 'ESR2', 'HIF1A'] 0.011603745748916518
4 / 1046 : ITGAM 9 ['ESR2', 'MYC', 'LEF1', 'JUN', 'ZEB1', 'ESR1', 'HIF1A', 'SMAD3', 'HLTF'] 0.021500602265324045
5 / 1046 : HMGXB4 10 ['GATA3', 'ZEB1', 'ESR1', 'MYC', 'HIF1A', 'SMAD3', 'JUN', 'ESR2', 'MECP2', 'CEBPB'] 0.011175157706070752
6 / 1046 : GPATCH2 10 ['MECP2', 'RXRA', 'BCL6', 'NR2C2', 'TAF1', 'IRF1', 'GATA3', 'LEF1', 'CLOCK', 'NFIA'] 0.007444889365938929
7 / 1046 : ZNF331 10 ['GATA3', 'LEF1', 'ZEB1', 'MYC', 'ESR1', 'SMAD3', 'HIF1A', 'JUN', 'MECP2', 'CEBPB'] 0.007435356166027052
8 / 1046 : NBEA 8 ['MYC', 'BACH2', 'ESR1', 'JUN', 'ESR2', 'SMAD3', 'MECP2', 'FOS'] 0.005118585613644713
9 / 1046 : RIPK2 9 ['MYC', 'ZEB1', 'LEF1', '

/Users/yijwang-admin/Documents/Research/GRN_PPI/L0GL_PPI_GRN/minFunc/polyinterp.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  t = points[notMinPos, 0] - (points[notMinPos, 0] - points[minPos, 0]) * ((points[notMinPos, 2] + d2 - d1) / (points[notMinPos, 2] - points[minPos, 2] + 2 * d2))


36 / 1046 : GRK3 10 ['ESR1', 'MYC', 'SMAD3', 'JUN', 'ZEB1', 'MECP2', 'ESR2', 'CEBPB', 'FOS', 'RXRA'] 0.010462222957973092


/Users/yijwang-admin/Documents/Research/GRN_PPI/L0GL_PPI_GRN/minFunc/polyinterp.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  t = points[notMinPos, 0] - (points[notMinPos, 0] - points[minPos, 0]) * ((points[notMinPos, 2] + d2 - d1) / (points[notMinPos, 2] - points[minPos, 2] + 2 * d2))


37 / 1046 : NFKBIA 10 ['ESR1', 'MYC', 'SMAD3', 'HIF1A', 'JUN', 'MECP2', 'CEBPB', 'ESR2', 'FOS', 'ZEB1'] 0.011604199614982788
38 / 1046 : HIVEP3 7 ['BACH2', 'ZEB1', 'ESR2', 'MYC', 'MAF', 'ESR1', 'JUN'] 0.010632459186201778
39 / 1046 : LIMS1 10 ['ESR2', 'MYC', 'ESR1', 'BACH2', 'HIF1A', 'SMAD3', 'JUN', 'MECP2', 'CEBPB', 'ZEB1'] 0.007574494672426435
40 / 1046 : SNAPC5 8 ['ESR2', 'MYC', 'ESR1', 'HIF1A', 'ZEB1', 'SMAD3', 'JUN', 'MECP2'] 0.008731364978430122
41 / 1046 : TAPT1 9 ['LEF1', 'ESR2', 'MYC', 'ESR1', 'JUN', 'SMAD3', 'MECP2', 'CEBPB', 'ETS1'] 0.007525906079940757
42 / 1046 : BCR 10 ['MYC', 'ESR1', 'HIF1A', 'SMAD3', 'JUN', 'ZEB1', 'MECP2', 'CEBPB', 'FOS', 'ESR2'] 0.006199287228620198
43 / 1046 : ICOS 9 ['MYC', 'ESR1', 'SMAD3', 'JUN', 'LEF1', 'CEBPB', 'ESR2', 'FOS', 'RXRA'] 0.00027260414451374767
44 / 1046 : PDSS2 10 ['ESR1', 'HIF1A', 'JUN', 'SMAD3', 'ESR2', 'MECP2', 'CEBPB', 'FOS', 'RXRA', 'NR4A1'] 0.00563466760130052
45 / 1046 : RPL11 5 ['ESR2', 'ZEB1', 'BACH2', 'SMAD3', 'FOXO3'] 0.01

...

806 / 1046 : AAMDC 9 ['ESR1', 'SMAD3', 'JUN', 'MECP2', 'CEBPB', 'ESR2', 'RXRA', 'NR4A1', 'BCL6'] 0.00611603436317362
807 / 1046 : CASC3 9 ['ESR1', 'MYC', 'HIF1A', 'ZEB1', 'SMAD3', 'JUN', 'MECP2', 'ETS1', 'CEBPB'] 0.0056357616639332005
808 / 1046 : PLAUR 10 ['GATA3', 'ESR2', 'ESR1', 'MYC', 'SMAD3', 'JUN', 'HIF1A', 'MECP2', 'CEBPB', 'RXRA'] 0.010619495539460148
809 / 1046 : SNX25 10 ['MYC', 'ESR1', 'HIF1A', 'SMAD3', 'JUN', 'MECP2', 'ZEB1', 'ESR2', 'GATA3', 'RXRA'] 0.0040294497124845905
810 / 1046 : ATP8B4 10 ['ESR2', 'ZEB1', 'LEF1', 'BACH2', 'ESR1', 'MYC', 'HIF1A', 'SMAD3', 'JUN', 'GATA3'] 0.010244441090792016
811 / 1046 : BCAS3 8 ['RORA', 'GATA3', 'ESR2', 'ESR1', 'MYC', 'JUN', 'SMAD3', 'MECP2'] 0.010080671950337452
812 / 1046 : RTN1 10 ['ZEB1', 'GATA3', 'JUN', 'MYC', 'ETS1', 'BACH2', 'ESR1', 'STAT4', 'ESR2', 'MECP2'] 0.02734016771225684
813 / 1046 : FCGR2A 10 ['MYC', 'GATA3', 'ESR2', 'FOS', 'ESR1', 'HIF1A', 'JUN', 'SMAD3', 'CEBPB', 'MECP2'] 0.015551631743043314
814 / 1046 : FGR 10 ['MYC

/Users/yijwang-admin/Documents/Research/GRN_PPI/L0GL_PPI_GRN/minFunc/polyinterp.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  t = points[notMinPos, 0] - (points[notMinPos, 0] - points[minPos, 0]) * ((points[notMinPos, 2] + d2 - d1) / (points[notMinPos, 2] - points[minPos, 2] + 2 * d2))


824 / 1046 : LAT 8 ['LEF1', 'ESR2', 'MYC', 'ESR1', 'JUN', 'SMAD3', 'HIF1A', 'MECP2'] 0.00170124581690683
825 / 1046 : GATA3 4 ['GATA3', 'MYC', 'ESR1', 'HIF1A'] 5.406054752642027e-07
826 / 1046 : LAP3 10 ['BACH2', 'ESR2', 'ZEB1', 'STAT4', 'MYC', 'ESR1', 'MECP2', 'SMAD3', 'HIF1A', 'JUN'] 0.016933796072879943
827 / 1046 : OASL 10 ['ZEB1', 'MYC', 'ESR2', 'ESR1', 'SMAD3', 'HIF1A', 'JUN', 'MECP2', 'RORA', 'CEBPB'] 0.015541462118362406
828 / 1046 : ABAT 9 ['MYC', 'ESR1', 'HIF1A', 'SMAD3', 'ZEB1', 'JUN', 'MECP2', 'CEBPB', 'LEF1'] 0.008473094783766205
829 / 1046 : PICALM 10 ['RORA', 'ESR2', 'ESR1', 'MYC', 'HIF1A', 'LEF1', 'SMAD3', 'JUN', 'MECP2', 'CEBPB'] 0.011987723733226735
830 / 1046 : NR4A2 3 ['NR4A2', 'ESR1', 'MYC'] 1.0104174226097073e-07
831 / 1046 : RAB11FIP1 10 ['MYC', 'ESR1', 'ZEB1', 'SMAD3', 'HIF1A', 'JUN', 'MECP2', 'CEBPB', 'ESR2', 'RXRA'] 0.00936880594906656
832 / 1046 : DYSF 10 ['MYC', 'ESR1', 'SMAD3', 'HIF1A', 'JUN', 'GATA3', 'ZEB1', 'ESR2', 'MECP2', 'CEBPB'] 0.0270373345539231
83

/Users/yijwang-admin/Documents/Research/GRN_PPI/L0GL_PPI_GRN/minFunc/polyinterp.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  t = points[notMinPos, 0] - (points[notMinPos, 0] - points[minPos, 0]) * ((points[notMinPos, 2] + d2 - d1) / (points[notMinPos, 2] - points[minPos, 2] + 2 * d2))


963 / 1046 : RPS14 10 ['ZEB1', 'MYC', 'ESR1', 'HIF1A', 'SMAD3', 'JUN', 'MECP2', 'CEBPB', 'ESR2', 'RXRA'] 0.013530128111949988
964 / 1046 : COLGALT1 10 ['LEF1', 'RORA', 'ETS1', 'GATA3', 'MYC', 'ESR1', 'JUN', 'HIF1A', 'SMAD3', 'MECP2'] 0.014098312838279576
965 / 1046 : B3GNTL1 9 ['ZEB1', 'ETS1', 'JUN', 'ESR2', 'ESR1', 'MYC', 'SMAD3', 'HIF1A', 'MECP2'] 0.010370530725247275
966 / 1046 : LUZP1 9 ['MYC', 'ESR1', 'HIF1A', 'SMAD3', 'JUN', 'LEF1', 'MECP2', 'ZEB1', 'CEBPB'] 0.008613363315179545
967 / 1046 : ZDHHC20 6 ['FOS', 'ZEB1', 'HIF1A', 'ESR1', 'JUN', 'SMAD3'] 0.011709440066085437
968 / 1046 : LYRM7 9 ['RORA', 'BACH2', 'ZEB1', 'MYC', 'ESR1', 'HIF1A', 'SMAD3', 'JUN', 'MECP2'] 0.0037525985325690234
969 / 1046 : PAX5 3 ['PAX5', 'ESR1', 'MYC'] 6.994903922798046e-07
970 / 1046 : SPAST 10 ['ESR2', 'LEF1', 'ESR1', 'ZEB1', 'JUN', 'HIF1A', 'SMAD3', 'MECP2', 'CEBPB', 'FOS'] 0.005686062196004372
971 / 1046 : TYMP 9 ['ESR1', 'MYC', 'ZEB1', 'HIF1A', 'SMAD3', 'JUN', 'MECP2', 'CEBPB', 'FOS'] 0.00645891341

In [30]:
GRIPNet_Res

,TG,TFs,Out_of_Sample MSE
0,XKR6,"[MYC, ESR1, JUN, SMAD3, ZEB1, MECP2, RXRA, ESR...",0.004189
1,MYO1F,"[MYC, ESR1, HIF1A, JUN, SMAD3, ESR2, MECP2, CE...",0.007562
2,B4GALT5,"[GATA3, RORA, ZEB1, FOS, MYC, JUN, ESR1, SMAD3...",0.011604
3,ITGAM,"[ESR2, MYC, LEF1, JUN, ZEB1, ESR1, HIF1A, SMAD...",0.021501
4,HMGXB4,"[GATA3, ZEB1, ESR1, MYC, HIF1A, SMAD3, JUN, ES...",0.011175
...,...,...,...
1021,IL3RA,"[ZEB1, RORA, LEF1, MYC, GATA3, BACH2, ESR1, ES...",0.011385
1022,MGAT4A,"[ESR1, MYC, HIF1A, SMAD3, JUN, MECP2, CEBPB, E...",0.003078
1023,TRPM2,"[ESR2, ZEB1, MYC, HIF1A, RORA, FOXO1, ESR1, ME...",0.011951
1024,TRAT1,"[ESR1, MYC, HIF1A, SMAD3, MECP2, ESR2, RXRA, N...",0.001033


In [29]:
GRIPNet_Res.to_pickle('PBMC3K_CD8_GRIP_Gene1846_Result_Motifp1e-4_lambda1e-3_pho1e-4_800_1846.pkl')

In [31]:
GRIPNet_Res_800 = pd.read_pickle('PBMC3K_CD8_GRIP_Gene1846_Result_Motifp1e-4_lambda1e-3_pho1e-4_0_800.pkl')

In [32]:
GRIPNet_Res_All = GRIPNet_Res._append(GRIPNet_Res_800, ignore_index=True)

In [33]:
GRIPNet_Res_All

,TG,TFs,Out_of_Sample MSE
0,XKR6,"[MYC, ESR1, JUN, SMAD3, ZEB1, MECP2, RXRA, ESR...",4.188703e-03
1,MYO1F,"[MYC, ESR1, HIF1A, JUN, SMAD3, ESR2, MECP2, CE...",7.562290e-03
2,B4GALT5,"[GATA3, RORA, ZEB1, FOS, MYC, JUN, ESR1, SMAD3...",1.160375e-02
3,ITGAM,"[ESR2, MYC, LEF1, JUN, ZEB1, ESR1, HIF1A, SMAD...",2.150060e-02
4,HMGXB4,"[GATA3, ZEB1, ESR1, MYC, HIF1A, SMAD3, JUN, ES...",1.117516e-02
...,...,...,...
1803,CCND3,"[ESR2, ZEB1, MYC, ESR1, HIF1A, SMAD3, JUN, BAC...",4.410861e-03
1804,NR4A1,"[NR4A1, MYC, ESR1]",6.176063e-08
1805,SMIM14,"[MYC, ESR2, ESR1, SMAD3, HIF1A, JUN, MECP2, CE...",7.358281e-03
1806,MX1,"[MYC, ZEB1, SMAD3, RORA, ESR1, HIF1A, LEF1, JU...",1.602978e-02


In [34]:
GRIPNet_Res_800

,TG,TFs,Out_of_Sample MSE
0,BCL2A1,"[GATA3, ESR2, STAT4, MYC, ZEB1, LEF1, JUN, BAC...",1.239409e-02
1,CBR4,"[MYC, ESR2, ESR1, JUN, SMAD3, HIF1A, MECP2, CE...",8.558710e-03
2,TCF7,"[TCF7, MYC, ESR1, HIF1A]",3.034622e-07
3,SLC24A4,"[MYC, ESR1, HIF1A, SMAD3, JUN, MECP2, CEBPB, Z...",1.823206e-02
4,FAM241A,"[GATA3, ZEB1, HIF1A, MAF, BACH1]",7.680549e-03
...,...,...,...
777,CCND3,"[ESR2, ZEB1, MYC, ESR1, HIF1A, SMAD3, JUN, BAC...",4.410861e-03
778,NR4A1,"[NR4A1, MYC, ESR1]",6.176063e-08
779,SMIM14,"[MYC, ESR2, ESR1, SMAD3, HIF1A, JUN, MECP2, CE...",7.358281e-03
780,MX1,"[MYC, ZEB1, SMAD3, RORA, ESR1, HIF1A, LEF1, JU...",1.602978e-02


In [35]:
GRIPNet_Res_All.to_pickle('PBMC3K_CD14_GRIP_Gene1808_Result_Motifp1e-4_lambda1e-3_pho1e-4.pkl')

In [ ]:
TGList = ['GNPTAB', 'MINK1', 'CAAP1', 'FGGY', 'CISH', 'PRIM2', 'DLG2', 'TTLL3', 'KLHL15', 'DNAJA1', 'SLC38A1', 'PPARD', 'LATS1', 'BLK', 'DDHD2', 'VPS45', 'N4BP2', 'KATNBL1', 'NEDD9', 'DUSP16', 'JARID2', 'DPP4', 'TGIF1', 'THEM4', 'CMIP', 'CREB5', 'MED19', 'PIK3CA', 'HEG1', 'IFT74', 'HSPA1A', 'SNX10', 'APLP2', 'TDP2', 'CD68', 'GSR', 'SELL', 'ZNF487', 'APC', 'SCNM1', 'IFITM1', 'MAP4K5', 'GCH1', 'PRDM1', 'POU2F1', 'ARAP2', 'IFI6', 'RCAN3', 'ZBTB16', 'BLOC1S4', 'KPNA5', 'PSMA1', 'GNB5', 'FLNB', 'CTDSPL2', 'VPS16', 'ITGA6', 'ASH2L', 'PIM1', 'NFRKB', 'HEATR3', 'CDIPT', 'KDM1B', 'NUP160', 'R3HDM1', 'MRPL28', 'SPECC1', 'CD5', 'RSPRY1', 'PHACTR4', 'LPCAT1', 'CCDC66', 'HOOK2', 'NFAM1', 'ANGPT2', 'NUDCD3', 'SIK3', 'PELI1', 'CTC1', 'TFEC', 'SSRP1', 'RPS6', 'SLAMF1', 'NFX1', 'NNT', 'FNDC3B', 'PTCD3', 'SPATA2L', 'CCND3', 'RIC8B', 'RPS15A', 'MTAP', 'MFHAS1', 'TMEM14B', 'AHNAK', 'RBM41', 'SNAPC5', 'INTS3', 'LRIG1', 'RMDN3', 'COQ7', 'UBA3', 'JAK2', 'CHD4', 'IL2RB', 'DGKA', 'SERTAD2', 'TSTD1', 'PTPRJ', 'ATP2B4', 'CSGALNACT1', 'C12orf57', 'ZRANB3', 'MRPL42', 'POLM', 'SNRK', 'PACS1', 'C11orf65', 'KCNQ1', 'ST6GAL1', 'HPS3', 'RPL31', 'SNAPC1', 'NUP107', 'SSBP3', 'CCDC167', 'MCM9', 'ABR', 'SLCO3A1', 'CEP68', 'ITGAE', 'PATL1', 'CHMP7', 'LRRC8C', 'AQP3', 'PDK1', 'LRP1', 'PRR5L', 'COA1', 'ING4', 'TTC39B', 'DCPS', 'ZDHHC5', 'SLC44A2', 'METTL21A', 'TET3', 'UBASH3B', 'CCAR2', 'IGF1R', 'ATXN1', 'CSPP1', 'FCER2', 'TESK2', 'LEPROTL1', 'IFT88', 'RIN3', 'DOCK8', 'ICAM1', 'TNFRSF1B', 'PIK3IP1', 'PRKCQ-AS1', 'SENP1', 'SCML4', 'CEACAM21', 'TNFAIP3', 'MCM3', 'LRRC23', 'TIPARP', 'HIVEP1', 'PSIP1', 'IMP3', 'IL21R', 'ITGAX', 'COTL1', 'SIK2', 'ZNF263', 'RBM10', 'SEH1L', 'TNIK', 'RPS14', 'INPP5K', 'RPS27', 'IL7R', 'CDC42EP3', 'ACTB', 'NUP43', 'PLIN2', 'ATP8A1', 'METTL16', 'TRERF1', 'SREK1IP1', 'TRABD2A', 'RHOH', 'ZMAT3', 'TBC1D14', 'HLA-DMB', 'BTBD2', 'APH1B', 'PIGL', 'AHR', 'COX10', 'FCGR2A', 'LCMT1', 'TTC4', 'TNFRSF10A', 'RELL1', 'LPCAT3', 'UXS1', 'COMMD9', 'ZBTB38', 'SYNE2', 'RPL3', 'CAMK1D', 'DHRS3', 'ODC1', 'NFATC2IP', 'NVL', 'IL4R', 'SLC30A7', 'LITAF', 'RPS26', 'TMEM238', 'AP4E1', 'PSMD6-AS2', 'VOPP1', 'ARHGEF11', 'SOS2', 'CD55', 'TDRD3', 'RAI1', 'UBE2E3', 'NUBPL', 'CD3E', 'KIF16B', 'CD247', 'AOAH', 'MT1X', 'DHRS4-AS1', 'ZNF789', 'SLC25A42', 'GATAD2B', 'E2F3', 'MAPKAPK2', 'ITGB1', 'MLLT3', 'GSE1', 'GBP5', 'ANKH', 'COG4', 'DHRS7B', 'NUAK2', 'RNF144A', 'HIVEP2', 'EZH2', 'SREBF1', 'CCND2', 'LAP3', 'BIRC3', 'OARD1', 'FGD2', 'CHIC2', 'CHPT1', 'FARS2', 'EMG1', 'RRM1', 'APBB1', 'CD27', 'LCORL', 'TMEM106B', 'ZFYVE28', 'TBL1X', 'RPS3', 'S100A4', 'IL6ST', 'LUZP1', 'ATR', 'UFC1', 'LY9', 'MPHOSPH9', 'SECISBP2L', 'HMGCS1', 'CX3CR1', 'ARHGEF18', 'FOXO1', 'DUSP6', 'SPTBN1', 'SFMBT2', 'LDLRAD4', 'DDX55', 'B4GALT1', 'RAB35', 'ATP13A1', 'UVSSA', 'TRNAU1AP', 'C9orf72', 'NUMB', 'RNF31', 'ITFG2', 'LIMK2', 'SLAMF7', 'SNX2', 'RCL1', 'RAB28', 'ETS1', 'RALGPS2', 'CBFB', 'VPS26B', 'NUP153', 'APOLD1', 'MPV17', 'GPATCH4', 'CLEC7A', 'GINM1', 'SYTL2', 'ENSA', 'CTNND1', 'ZFX', 'FTL', 'NFKBIZ', 'TFRC', 'SLC35E3', 'WBP1', 'GNL1', 'SIAH1', 'APBA2', 'FHIT', 'SESN3', 'WDR59', 'HLA-DRA', 'NXPE3', 'P2RX1', 'MCPH1', 'CD69', 'CD6', 'APIP', 'SLC12A6', 'KRI1', 'CDR2', 'LMNA', 'GPCPD1', 'FOS', 'GLCCI1', 'HK2', 'ARHGAP5', 'VCPKMT', 'TXK', 'ITPKB', 'MCCC1', 'BHLHE40', 'TRDMT1', 'NCAPD2', 'SUPT3H', 'TMEM204', 'PRMT3', 'CREBL2', 'MLH1', 'SERPINB9', 'MYLIP', 'DUS2', 'NPAT', 'AP3M2', 'DHX16', 'RBL2', 'NEAT1', 'SPTY2D1', 'SLMAP', 'RRBP1', 'STK17A', 'PEX1', 'SLAMF6', 'PAM16', 'TRANK1']


GRIPNet_Res = Run_GRIPNet(CD4_scRNA_DN_magic_ad, SID_train, SID_outsample, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, G_PPI_BioGrid, TF_Motif_Metric[2], TargetGenes = TGList)

In [ ]:
GRIPNet_Res, = Run_GRIPNet(CD4_scRNA_DN_magic_ad, SID_train, SID_outsample, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, G_PPI_BioGrid, TF_Motif_Metric[1], TargetGenes = TGList)

In [ ]:
Gene_in_CD4_scRNA.index('TMLHE-AS1')

In [ ]:
Fname = "./CD4_GRIP_Result_Tmp.txt"

GRIPNet_DF = pd.DataFrame(columns=['TG', 'TFs', 'Out_of_Sample MSE'])
fh = open(Fname, 'r')
for line in fh:
    linet = line.strip().split(' ')
    if linet[0] == 'Fail':
        continue
    else:
        TT = line.split(' ')
        Str = ''
        for i in range(2, len(TT)-1):
            #print(TT[i])
            Str = Str + TT[i]
        TFList = eval(Str)
        #print(linet[-1])
        GRIPNet_DF = GRIPNet_DF._append({'TG': linet[0], 'TFs': TFList, 'Out_of_Sample MSE': float(linet[-1])}, ignore_index=True)

        


In [ ]:
GRIPNet_DF

In [ ]:
Fname = "./CD4_GRIP_Result_Motifp1e-4_lambda1e-4_pho1e-4.txt"

GRIPNet_DF = pd.DataFrame(columns=['TG', 'TFs', 'Out_of_Sample MSE'])
fh = open(Fname, 'r')
for line in fh:
    linet = line.strip().split(' ')
    if linet[0] == 'Fail':
        continue
    else:
        TT = line.split(' ')
        Str = ''
        for i in range(2, len(TT)-1):
            #print(TT[i])
            Str = Str + TT[i]
        TFList = eval(Str)
        #print(linet[-1])
        GRIPNet_DF = GRIPNet_DF._append({'TG': linet[0], 'TFs': TFList, 'Out_of_Sample MSE': float(linet[-1])}, ignore_index=True)

        

In [ ]:
GRIPNet_DF

In [ ]:
GRIPNet_DF.to_pickle('PBMC3K_CD4_GRIP_Result_Motifp1e-4_lambda1e-4_pho1e-4.pkl')

In [ ]:
TF_Inferred = list(set(itertools.chain.from_iterable(GRIPNet_DF['TFs'].tolist())))
print(len(TF_Inferred))

In [ ]:
GRIPNet_DF.to_pickle('PBMC3K_CD4_GRIP_Result.pkl')

In [ ]:
Tmp_Input = pbmc_3k_scATAC_Gene_Enhancer_CD4_psum[pbmc_3k_scATAC_Gene_Enhancer_CD4_psum['Genes_within_250k'].apply(lambda x: any(gene in x for gene in ['RPS27']))].reset_index()



In [ ]:
Tmp_Input

## 1.3 Load PCHiC ground truth and trim to CD4

In [ ]:
#load the data
PCHiC_Overlap_Pbmc_3k_AllOP = pd.read_pickle('../../Data/GroundTruth/pbmc_3k_PCHiC/PCHiC_Overlap_Pbmc_3k_ATAC_All_OpenRegion.pkl')


In [ ]:
PCHiC_Overlap_Pbmc_3k_AllOP

In [ ]:
print(list(PCHiC_Overlap_Pbmc_3k_AllOP.columns)[-15:])

In [ ]:
Df_tmp = PCHiC_Overlap_Pbmc_3k_AllOP[['chr_position', 'nCD4', 'baitNameList', 'oeChr', 'oeStart', 'oeEnd', 'Hit']]
Df_tmp = Df_tmp[Df_tmp['nCD4']>=5]
CD4_PCHiC = Df_tmp[Df_tmp['Hit']>=1]

In [ ]:
CD4_PCHiC

In [ ]:
Gene_in_PCHiC = DF_Column_to_List(CD4_PCHiC, 'baitNameList')
print(len(Gene_in_PCHiC))

In [ ]:
OP_Overlap_PCHiC = DF_Column_to_List(CD4_PCHiC, 'chr_position')
print(len(OP_Overlap_PCHiC))

Gene_with_GT = list()
for op in OP_Overlap_PCHiC:
    Gene_with_GT.append(op.split('_')[0])

In [ ]:
Gene_in_CD4_scATAC_scRNA_PCHiC = list(set(Gene_with_GT) & set(Gene_in_CD4_scATAC_scRNA))
print('Gene appears in both scRNA, scATAC, and PCHiC is', len(Gene_in_CD4_scATAC_scRNA_PCHiC))

## 1.4 Load PPI

In [ ]:
G_PPI_BioGrid = pickle.load(open('../../Data/PPI/PPI_BioGrid_Evd2_DSD.pickle', 'rb'))
print("The PPI netowrk has", len(G_PPI_BioGrid.nodes), "nodes and", len(G_PPI_BioGrid.edges), "edges")

# 2. Prepare data for Alg.

In [ ]:
Gene_scRNA = CD4_scRNA_DN_magic_ad.var.index.tolist()
print(len(Gene_scRNA))

# intersection between genes in scRNA and PCHiC ground truth
Gene_Validate = Gene_in_CD4_scATAC_scRNA_PCHiC

print("We focus on", len(Gene_Validate), "genes")

In [ ]:
TargetG = Gene_Validate[1]
print(TargetG)

In [ ]:
Gene_with_GT = list()
Map_Gene_GTOpenR = {}
for i in range(len(Gene_Validate)):
    GL = [Gene_Validate[i]]
    #print(GL)
    Tmp_Input = pbmc_3k_scATAC_Gene_Enhancer_CD4_psum[pbmc_3k_scATAC_Gene_Enhancer_CD4_psum['Genes_within_250k'].apply(lambda x: any(gene in x for gene in GL))].reset_index()

    Tmp_GT = CD4_PCHiC[CD4_PCHiC['baitNameList'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
    
    GTtmp =  Tmp_GT['chr_position'].tolist()
    Input_tmp = Tmp_Input['chr_position'].tolist()

    NumGT = 0
    OPList = list()
    for tt in GTtmp:
        tsp = tt[0].split('_', 1)
        chrp = tsp[1]#+'_'+tsp[2]+'_'+tsp[3]
        #print(tsp, chrp)
        if tsp[0] == GL[0] and chrp in Input_tmp:
            NumGT = NumGT + 1
            OPList.append(chrp)
    
    if NumGT > 0:
        Gene_with_GT.append(GL[0])
        Map_Gene_GTOpenR[GL[0]] = list(set(OPList))
        print(GL[0], "Num OP is", Tmp_Input.shape[0], "Num GT in those OP is", len(set(OPList)))



In [ ]:
print(list(Map_Gene_GTOpenR.keys()))


In [ ]:
# Set up Objective Function
funObj = lambda w: L0GLObj(w, X, y, pho)

funObj_PPI = lambda w: L0GL_PPI_Obj(w, X, y, S, pho, lamb)

# Set up Simplex Projection Function
funProj = lambda w: ProjCSimplex(w, k)

funProj_GL = lambda w: ProjCSimplexGL_Gurobi(w, k, Group, h)

funProj_GL_LB = lambda w: ProjCSimplexGL_Gurobi_LB(w, k, Group, h)

In [ ]:
ColSelect = 'TF_Matching_p-value_1e-05'
lamb = 0.01
pho = 0.01#np.sqrt(NumSample)
h = 1
#Group = [range(NumFeature)]#GroupInd#
options = {'maxIter': 400, 'verbose': 0}

Num_Our_better = 0
for gt in Gene_with_GT:
    GL = [gt]
    TF_Group, TmpInput = Get_TFs_in_Data_noPrint(gt, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, Gene_scRNA)
    
    TFList_ = list(set(list(itertools.chain.from_iterable(TF_Group))))#DF_Column_to_List(Tmp_Input, ColSelect)
    TFList_in_Data = list(set(TFList_) & set(Gene_scRNA))
    
    NumFeature = len(TFList_in_Data)
    NumSample = CD4_scRNA_DN_magic_ad[:,TFList_in_Data].X.shape[0]
    Group = [range(NumFeature)]
    #print(NumFeature)

    # Input data
    Xt = CD4_scRNA_DN_magic_ad[:,TFList_in_Data].X
    yt = CD4_scRNA_DN_magic_ad[:,GL].X
    SampleIDs = CD4_scRNA_DN_magic_ad[:,TFList_in_Data].obs.index.tolist()

    uSimplex = np.ones((NumFeature, 1)) * (1 / NumFeature)
    #print(uSimplex.shape)

    S =  Short_Path_DSD_Matrix(G_PPI_BioGrid, TFList_in_Data)#Short_Path_Matrix(G_PPI_BioGrid, TFList_in_Data)

    LSE_Tmp_TFGraph = np.zeros(5)
    LSE_Tmp_No_TFGraph = np.zeros(5)
    k = 4#int(0.1*NumFeature)
    for t in range(5):
        X, y, X_outsample, y_outsample, _, _ = Split_Sample(Xt, yt, SampleIDs, 0.9)
    
        # parameters that can be changed
        lamb = 0.01
        
        uout, obj, _ = minConF_PQN(funObj_PPI, uSimplex, funProj_GL_LB, options)
    
        uout_ppi, zout_ppi = ProjCSimplexGL_Gurobi_LB_v2(uout, k, Group, h)
        #print(zout_ppi.flatten())
        SelectID = Extract_Solution(uout_ppi, zout_ppi, Group, k, h)
        
        ut = np.zeros((NumFeature,1))
        ut[SelectID,0] = 1.0

        ObjF, _  = L0GL_PPI_Obj(ut, X, y, S, pho, lamb)
        #print("Obj:", ObjF)
        #print("Short Dis;", ut.T@S@ut)


        XSelect = X[:,SelectID]
        w_est = np.linalg.pinv(XSelect.T @ XSelect + pho*np.eye(len(SelectID))) @ XSelect.T @ y
        #print(TF_Select)
        #print(np.abs(w_est.flatten()))

        X_outsample_select = X_outsample[:, SelectID]
        LSE_Tmp_TFGraph[t] = np.linalg.norm(y_outsample - X_outsample_select @ w_est)
    
        # parameters that can be changed
        lamb = 0.0
    
        uout, obj, _ = minConF_PQN(funObj_PPI, uSimplex, funProj_GL_LB, options)
    
        uout_ppi, zout_ppi = ProjCSimplexGL_Gurobi_LB_v2(uout, k, Group, h)
        #print(zout_ppi.flatten())
        SelectID = Extract_Solution(uout_ppi, zout_ppi, Group, k, h)
        
        ut = np.zeros((NumFeature,1))
        ut[SelectID,0] = 1.0

        ObjF, _  = L0GL_PPI_Obj(ut, X, y, S, pho, lamb)
        #print("Obj:", ObjF)
        #print("Short Dis;", ut.T@S@ut)


        XSelect = X[:,SelectID]
        w_est = np.linalg.pinv(XSelect.T @ XSelect + pho*np.eye(len(SelectID))) @ XSelect.T @ y
        #print(TF_Select)
        #print(np.abs(w_est.flatten()))

        X_outsample_select = X_outsample[:, SelectID]
        LSE_Tmp_No_TFGraph[t] = np.linalg.norm(y_outsample - X_outsample_select @ w_est)
    

    if np.mean(LSE_Tmp_TFGraph) >= np.mean(LSE_Tmp_No_TFGraph):
        print('-', gt, np.mean(LSE_Tmp_TFGraph), np.std(LSE_Tmp_TFGraph),  np.mean(LSE_Tmp_No_TFGraph), np.std(LSE_Tmp_No_TFGraph))
    else:
        print('+', gt, np.mean(LSE_Tmp_TFGraph), np.std(LSE_Tmp_TFGraph),  np.mean(LSE_Tmp_No_TFGraph), np.std(LSE_Tmp_No_TFGraph))
    
    
    if np.mean(LSE_Tmp_TFGraph) < np.mean(LSE_Tmp_No_TFGraph):
        Num_Our_better = Num_Our_better + 1

print(Num_Our_better," out of ", len(Gene_with_GT), "our new model is better!")

In [ ]:
TG = 'TIPARP'
ColSelect = 'TF_Matching_p-value_1e-05'

GL = [TG]

TF_Group, TmpInput = Get_TFs_in_Data(TG, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, Gene_scRNA)
#print(TF_Group)

TFList_ = list(set(list(itertools.chain.from_iterable(TF_Group))))#DF_Column_to_List(Tmp_Input, ColSelect)
TFList_in_Data = list(set(TFList_) & set(Gene_scRNA))
print(TFList_)
print(TFList_in_Data)

GroupInd = Group_Ind(TFList_in_Data, TF_Group)
#print(GroupInd)

{'CEBPD', 'FOS', 'KLF4'}

In [ ]:
NumFeature = len(TFList_in_Data)
NumSample = CD4_scRNA_DN_magic_ad[:,TFList_in_Data].X.shape[0]
print(NumFeature)

# Input data
Xt = CD4_scRNA_DN_magic_ad[:,TFList_in_Data].X
yt = CD4_scRNA_DN_magic_ad[:,GL].X
SampleIDs = CD4_scRNA_DN_magic_ad[:,TFList_in_Data].obs.index.tolist()

X, y, X_outsample, y_outsample, SampleID_train, SampleID_outofsample = Split_Sample(Xt, yt, SampleIDs, 0.9)

uSimplex = np.ones((NumFeature, 1)) * (1 / NumFeature)
print(uSimplex.shape)

S =  Short_Path_DSD_Matrix(G_PPI_BioGrid, TFList_in_Data)#Short_Path_Matrix(G_PPI_BioGrid, TFList_in_Data)
#ShortPath = ShorestPath_GivenTFs(G_PPI_BioGrid, TFList_in_Data, "TestShortPath.txt")
print(S.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
plt.imshow(S, cmap='hot')
plt.xticks(range(len(TFList_in_Data)), TFList_in_Data)
plt.colorbar()
plt.show()


In [ ]:
lamb = 0.001
pho = 0.000001#np.sqrt(NumSample)
h = 1
Group = [range(NumFeature)]#GroupInd#

k = 6
h = 1
      
# parameters that can be changed
options = {'maxIter': 400, 'verbose': 1}

K_Range = range(3,10)
MSE_Our = np.zeros(len(K_Range))
for i in range(len(K_Range)):
    k = K_Range[i]
    
    uout, obj, _ = minConF_PQN(funObj_PPI, uSimplex, funProj_GL_LB, options)
    
    uout_ppi, zout_ppi = ProjCSimplexGL_Gurobi_LB_v2(uout, k, Group, h)
    #print(zout_ppi.flatten())
    SelectID = Extract_Solution(uout_ppi, zout_ppi, Group, k, h)

    TF_Select = list()
    for idt in SelectID:
        TF_Select.append(TFList_in_Data[idt])
    print(TF_Select)


    MSE_Our[i] = Out_of_Sample_MSE(CD4_scRNA_DN_magic_ad, TG, TF_Select, SampleID_train, SampleID_outofsample)
    print(k, MSE_Our[i])


In [ ]:
TF_Select_scenic = ['CREB5', 'BCL11B', 'KLF4', 'SPI1', 'FOS']
print(TF_Select_scenic)
MSE_scenic = Out_of_Sample_MSE(CD4_scRNA_DN_magic_ad, TG, TF_Select_scenic, SampleID_train, SampleID_outofsample)
print(MSE_scenic)

In [ ]:
import pwlf

my_pwlf = pwlf.PiecewiseLinFit(K_Range, MSE_Our)
breaks = my_pwlf.fit(2)
print(breaks)

x_hat = np.linspace(K_Range[0], K_Range[-1], 100)
y_hat = my_pwlf.predict(x_hat)

plt.figure()
plt.plot(K_Range, MSE_Our, 'o')
plt.plot(x_hat, y_hat, '-')
plt.show()

In [ ]:
from scipy import optimize
def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x <= x0], [lambda x:k1*x + y0+k1*x0, lambda x:k1*x0 + y0-k1*x0])

p , e = optimize.curve_fit(piecewise_linear, K_Range, MSE_Our)
print(p)

In [ ]:
xd = np.linspace(3, 9, 100)
plt.plot(K_Range, MSE_Our, "o")
plt.plot(xd, piecewise_linear(xd, *p))

In [ ]:
TF_Select = list()
for idt in SelectID:
    TF_Select.append(TFList_in_Data[idt])
print(TF_Select)
#print(SelectID_Best)
TF_Group = Show_TFs_in_Data(TG, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, TF_Select)

TF_Select = TF_Select_scenic
print(TF_Select)
TF_Group = Show_TFs_in_Data(TG, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, TF_Select)

In [ ]:
GS = GL  + (TF_Select)
yt = CD4_scRNA_DN_magic_ad[:,GS].X

fig, ax = plt.subplots(nrows=len(GS), ncols=1, sharex=True, figsize=(12, 10))
for i in range(len(GS)):
    ax[i].plot(yt[:,i])
    ax[i].set_title(GS[i])


In [ ]:
St =  Short_Path_DSD_Matrix(G_PPI_BioGrid, ['IRF8', 'MEF2C', 'HES1'])#Short_Path_Matrix(G_PPI_BioGrid, TFList_in_Data)


fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(St, cmap='hot')
plt.xticks(range(len(TF_Select)), TF_Select)
plt.colorbar()
plt.show()

In [ ]:
lamb = 0.0
pho = 0.1#np.sqrt(NumSample)
h = 1
Group = [range(NumFeature)]#GroupInd#

uSimplex = np.ones((NumFeature, 1)) * (1 / NumFeature)
print(uSimplex.shape)

S = Short_Path_Matrix(G_PPI_BioGrid, TFList_in_Data)
ShortPath = ShorestPath_GivenTFs(G_PPI_BioGrid, TFList_in_Data, "TestShortPath.txt")
print(S.shape)

NumItr = 10
k_range = range(3,10)
#k = 6
h_range = range(1,2)
SelectID_Best = ()
LSE_best = 10000
for k in k_range:
    for h in h_range:
        LSE_Tmp = np.zeros(NumItr)
        for i in range(NumItr):
            X, y, X_outsample, y_outsample = Split_Sample(Xt, yt, 0.9)
            
            # parameters that can be changed
            options = {'maxIter': 400, 'verbose': 1}
    
            uout, obj, _ = minConF_PQN(funObj_PPI, uSimplex, funProj_GL_LB, options)
    
            uout_ppi, zout_ppi = ProjCSimplexGL_Gurobi_LB_v2(uout, k, Group, h)
            #print(zout_ppi.flatten())
            SelectID = Extract_Solution(uout_ppi, zout_ppi, Group, k, h)
        
            ut = np.zeros((NumFeature,1))
            ut[SelectID,0] = 1.0

            ObjF, _  = L0GL_PPI_Obj(ut, X, y, S, pho, lamb)
            #print("Obj:", ObjF)
            #print("Short Dis;", ut.T@S@ut)


            XSelect = X[:,SelectID]
            w_est = np.linalg.pinv(XSelect.T @ XSelect + pho*np.eye(len(SelectID))) @ XSelect.T @ y
            #print(TF_Select)
            #print(np.abs(w_est.flatten()))

            X_outsample_select = X_outsample[:, SelectID]
            LSE_Tmp[i] = np.linalg.norm(y_outsample - X_outsample_select @ w_est)
        LSE = np.mean(LSE_Tmp)    
        print('k=',k, 'h=', h,"Out-of-sample LSE:", LSE)
    
        if LSE < LSE_best:
            LSE_best = LSE
            SelectID_Best = SelectID
    
    

In [ ]:
TF_Select = list()
for idt in SelectID_Best:
    TF_Select.append(TFList_in_Data[idt])
print(TF_Select)
print(SelectID_Best)

TF_Group = Show_TFs_in_Data(TG, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, TF_Select)

In [ ]:
TF_Select = list()
for idt in SelectID_Best:
    TF_Select.append(TFList_in_Data[idt])
print(TF_Select)
print(SelectID_Best)

TF_Group = Show_TFs_in_Data(TG, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, TF_Select)

In [ ]:
uout, obj, _ = minConF_PQN(funObj_PPI, uSimplex, funProj_GL_LB, options)

In [ ]:
uout_ppi, zout_ppi = ProjCSimplexGL_Gurobi_LB_v2(uout, k, Group, h)
print(zout_ppi.flatten())
SelectID = Extract_Solution(uout_ppi, zout_ppi, Group, k, h)


TF_Select = list()
for idt in SelectID:
    TF_Select.append(TFList_in_Data[idt])
print(TF_Select)
print(SelectID)

In [ ]:
TF_Group = Show_TFs_in_Data(TG, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, TF_Select)

In [ ]:
ut = np.zeros((NumFeature,1))
ut[SelectID,0] = 1.0

ObjF, _  = L0GL_PPI_Obj(ut, X, y, S, pho, lamb)
print("Obj:", ObjF)
print("Short Dis;", ut.T@S@ut)


XSelect = X[:,SelectID]
w_est = np.linalg.pinv(XSelect.T @ XSelect) @ XSelect.T @ y
print(TF_Select)
print(np.abs(w_est.flatten()))

X_outsample_select = X_outsample[:, SelectID]
LSE = np.linalg.norm(y_outsample - X_outsample_select @ w_est)
print("Out-of-sample LSE:", LSE)

In [ ]:
plt.plot(y_outsample)
plt.plot(X_outsample_select @ w_est)

In [ ]:
uout_ppi = ProjCSimplexGL_Gurobi_LB(uout, k, Group, h)

In [ ]:
np.argsort(-zout_ppi.flatten())

In [ ]:
zout_ppi.flatten()

In [ ]:
Ind = np.argsort(-uout_ppi.flatten())[0:30]
print(Ind)
TFRes = list()
for idt in Ind:
    #print(TFList_in_Data[idt])
    TFRes.append(TFList_in_Data[idt])
print(TFRes)

In [ ]:
TFList_Raw = Tmp_Input['TF_Matching_p-value_1e-05'].tolist()
Chr_Position_Raw = Tmp_Input['chr_position'].tolist()
count = 0
for tfl, chp in zip(TFList_Raw, Chr_Position_Raw):
    if chp in Map_Gene_GTOpenR[GL[0]]:
        hit = 1
    else:
        hit = 0
    olp = list(set(tfl) & set(TFRes))
    print(count, olp, hit)
    count = count + 1

In [ ]:
uout.T @ S @ uout

In [ ]:
ShortPath = ShorestPath_GivenTFs(G_PPI_BioGrid, TFRes, "TestShortPath_Result.txt")
print(ShortPath)

# Tmp

In [ ]:
Tmp_Input

In [ ]:
Tmp_GT

In [ ]:
Tmp_Input

In [ ]:
tsp

In [ ]:
TFList_Raw = Tmp['TFs_binding'].tolist()
Hit_Raw = Tmp['Hit'].tolist()
for tfl,hit in zip(TFList_Raw, Hit_Raw):
    olp = list(set(tfl) & set(Gene_scRNA))
    print(olp, hit)

In [ ]:
Tmp['baitNameList'][0]

In [ ]:
for gt in Gene_Validate:
    GL = [gt]
    #print(GL)
    Tmp = CD4_scATAC_Gene_TF_Enhancer_PCHiC_Trim[CD4_scATAC_Gene_TF_Enhancer_PCHiC_Trim['Genes_within_250k'].apply(lambda x: any(gene in x for gene in GL))].reset_index()

    TF_Candiates =  list(set(list(itertools.chain.from_iterable(Tmp['TFs_binding'].tolist()))))
    #print(TF_Candiates)

    TF_In_Data = list(set(Gene_scRNA) & set(TF_Candiates))
    #print(TF_In_Data)
    
    Num_GT = Tmp[~Tmp['baitNameList'].isnull()]['baitNameList'].apply(lambda x: any(gene in x for gene in GL)).sum()
    
    print(gt, "Total_TF / TF_in_Data = ", len(TF_Candiates), "/", len(TF_In_Data), "GT number:", Num_GT)
    

In [ ]:
CD4_scATAC_Gene_TF_Enhancer[['chr', 'start', 'end']] = CD4_scATAC_Gene_TF_Enhancer['chr_position'].str.split('_', expand=True)

In [ ]:
CD4_scATAC_Gene_TF_Enhancer

In [ ]:

GL = [Gene_Validate[0]]
    #print(tg)
Tmp = CD4_scATAC_Gene_TF_Enhancer[CD4_scATAC_Gene_TF_Enhancer['output_grch38_col4'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
TG_TFList = list(set(list(itertools.chain.from_iterable(Tmp['output_fimo_col4'].tolist()))))
    #print("TFs for the target is", len(TG_TFList))

TG_TGList_InData = list(set(TG_TFList) & set(Gene_scRNA))
    #print("TFs for the target in the data is", len(TG_TGList_InData))


In [ ]:
Tmp

In [ ]:
GeneCount = 0
TruthRegion = 0
Gene_with_GroundTruth = list()
True_OpenRegin_List = list()
for tg in Gene_Validate:
    GL = [tg]
    Tmp = CD4_scATAC_Gene_TF_Enhancer[CD4_scATAC_Gene_TF_Enhancer['output_grch38_col4'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
    TG_TFList = list(set(list(itertools.chain.from_iterable(Tmp['output_fimo_col4'].tolist()))))
    #print("TFs for the target is", len(TG_TFList))

    TG_TGList_InData = list(set(TG_TFList) & set(Gene_scRNA))
    #print("TFs for the target in the data is", len(TG_TGList_InData))

    Tmp = Tmp[Tmp['sum']>=1]
    #print(Tmp)

    CD4_Target = CD4_PCHiC[CD4_PCHiC['baitNameList'].apply(lambda lst: tg in lst) ]

    Start_val = Tmp['start'].values.astype(int)
    End_val = Tmp['end'].values.astype(int)

    oeStart_val = CD4_Target['oeStart'].values
    oeEnd_val = CD4_Target['oeEnd'].values

    start_mask = np.logical_and(
        Start_val[:, np.newaxis] >= oeStart_val-500,
        Start_val[:, np.newaxis] <= oeEnd_val+500
    )

    end_mask = np.logical_and(
        End_val[:, np.newaxis] >= oeStart_val-500,
        End_val[:, np.newaxis] <= oeEnd_val+500
    )


    HitList=np.array(np.logical_or(start_mask, end_mask).sum(axis=1))
    #print(HitList)
    Ind = np.where(HitList)[0]
    #print(Ind)
    #print(Tmp['chr_position'])
    #print(Tmp['chr_position'][Ind].values)
    OP_Hit_List = Tmp['chr_position'][Ind].values
    #print(OP_Hit_List)
    
    Overlap_regions = np.logical_or(start_mask, end_mask).any(axis=1).sum()
    if Overlap_regions > 0:
        GeneCount = GeneCount + 1
        TruthRegion = TruthRegion + Overlap_regions
        Gene_with_GroundTruth.append(tg)
        #print(tg)
        #print("TFs for the target is", len(TG_TFList))
        #print("TFs for the target in the data is", len(TG_TGList_InData))
        print(tg, "open reigions(", Tmp.shape[0], ")overlaps with", Overlap_regions, "within the PCHiC", CD4_Target.shape[0], "TF_Raw/TF_Exp=", len(TG_TFList), "/", len(TG_TGList_InData))
        True_OpenRegin_List.extend(OP_Hit_List)
        #print(OP_Hit_List[0])
        

In [ ]:
print("Gene:", len(Gene_with_GroundTruth), "Links", TruthRegion)

In [ ]:
len(set(True_OpenRegin_List))

In [ ]:
CD4_scATAC_Gene_TF_Enhancer['Hit_PCHiC'] = 0

In [ ]:
CD4_scATAC_Gene_TF_Enhancer['Hit_PCHiC'] = CD4_scATAC_Gene_TF_Enhancer['chr_position'].apply(lambda x: 1 if x in True_OpenRegin_List else 0)

In [ ]:
CD4_scATAC_Gene_TF_Enhancer['chr_position'].apply(lambda chrp: chrp in True_OpenRegin_List)['Hit_PCHiC'] = 1

In [ ]:
CD4_scATAC_Gene_TF_Enhancer

In [ ]:
CD4_scATAC_Gene_TF_Enhancer[CD4_scATAC_Gene_TF_Enhancer['chr_position'] == True_OpenRegin_List[0]]

In [ ]:
Tmp = CD4_scATAC_Gene_TF_Enhancer[CD4_scATAC_Gene_TF_Enhancer['output_grch38_col4'].apply(lambda x: any(gene in x for gene in ['MAPKAPK2']))].reset_index()


In [ ]:
Tmp

In [ ]:
Tmp_TFs = Tmp['output_fimo_col4'].tolist()
for tfl in Tmp_TFs:
    Int = list(set(TG_TGList_InData) & set(tfl))
    print(sorted(Int))

In [ ]:
sp = nx.shortest_path_length(G_PPI_BioGrid, 'PAX5', 'SP2')
print(sp)
Path = nx.shortest_path(G_PPI_BioGrid, source='PAX5', target='SP2')
print(Path)

In [ ]:
Tmp

In [ ]:
Tmp[['chr', 'start', 'end']] = Tmp['chr_position'].str.split('_', expand=True)

In [ ]:
Tmp

In [ ]:
CD4_Target = CD4_PCHiC[CD4_PCHiC['baitNameList'].apply(lambda lst: TargetG in lst) ]

In [ ]:
Start_val = Tmp['start'].values.astype(int)
End_val = Tmp['end'].values.astype(int)

oeStart_val = CD4_Target['oeStart'].values
oeEnd_val = CD4_Target['oeEnd'].values

In [ ]:
start_mask = np.logical_and(
        Start_val[:, np.newaxis] >= oeStart_val,
        Start_val[:, np.newaxis] <= oeEnd_val
    )

end_mask = np.logical_and(
        End_val[:, np.newaxis] >= oeStart_val,
        End_val[:, np.newaxis] <= oeEnd_val
    )

Overlap_regions = np.logical_or(start_mask, end_mask).any(axis=1).sum()

In [ ]:
Overlap_regions

In [ ]:
CD4_Target

In [ ]:
int(gene_tss[gene_tss['genes'] == TargetG]['tss_corrected'].values[0])+250000, int(gene_tss[gene_tss['genes'] == TargetG]['tss_corrected'].values[0])-250000

In [ ]:
print(Tmp['output_fimo_col4'].tolist())

In [ ]:
GL